<a href="https://colab.research.google.com/github/HasanAbdallah12/CloudCourse/blob/main/mqtt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""mqtt.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1vKaPz0gDXWkRtjD_Ey09DX3hLUuQL1TN
"""

# @title Imports and Global Variables

# Fix for collections issue in Python 3.10+
import collections.abc
collections.MutableMapping = collections.abc.MutableMapping
collections.Mapping = collections.abc.Mapping
collections.Iterable = collections.abc.Iterable

import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict, Counter
import json
import time
from IPython.display import display, HTML, clear_output, Javascript
import ipywidgets as widgets
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
!pip install paho-mqtt
# !pip install plotly
import paho.mqtt.client as mqtt
import threading
from queue import Queue
import random

# Add NLTK imports with proper download handling
import nltk

# Download required NLTK data
print("Downloading NLTK data...")
nltk.download('wordnet', quiet=True)
nltk.download('words', quiet=True)
nltk.download('punkt', quiet=True)

# Now import the required modules
try:
    from nltk.corpus import wordnet
    from nltk.corpus import words

    # Create a set of valid English words
    english_words = set(words.words())
    use_nltk = True
    print("NLTK word validation ready!")
except Exception as e:
    print(f"NLTK setup failed: {e}")
    print("Using basic word validation instead")
    english_words = set()
    use_nltk = False

# Firebase Configuration
FIREBASE_URL = "https://mqtt-eeec6-default-rtdb.firebaseio.com/"

# These values are only for reference - we won't connect to MQTT
#MQTT_BROKER = "test.mosquitto.org"
#MQTT_PORT = 1883
#INDOOR_TOPIC = "braude/D106/indoor"
#OUTDOOR_TOPIC = "braude/D106/outdoor"

# Enhanced styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")


print("All imports successful!")
import sys
# Redirect stdout to null device
sys.stdout = open(os.devnull, 'w')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
# @title Style CSS

# Enhanced CSS with glassmorphism and modern animations
premium_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700;900&display=swap');

* {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
    box-sizing: border-box;
}

body {
    background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 50%, #e0c3fc 100%);
    min-height: 100vh;
}

.app-container {
    background: rgba(255, 255, 255, 0.15);
    backdrop-filter: blur(20px);
    border: 1px solid rgba(255, 255, 255, 0.3);
    border-radius: 30px;
    padding: 50px;
    margin: 20px auto;
    box-shadow: 0 25px 50px rgba(0, 0, 0, 0.1);
    animation: slideIn 0.8s cubic-bezier(0.68, -0.55, 0.265, 1.55);
}

@keyframes slideIn {
    from {
        opacity: 0;
        transform: translateY(50px) scale(0.9);
    }
    to {
        opacity: 1;
        transform: translateY(0) scale(1);
    }
}

.main-title {
    background: linear-gradient(45deg, #6c5ce7 0%, #a29bfe 25%, #74b9ff 50%, #0984e3 75%, #00cec9 100%);
    background-size: 200% 200%;
    background-clip: text;
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 72px;
    font-weight: 900;
    text-align: center;
    margin-bottom: 20px;
    animation: gradient 3s ease infinite;
    letter-spacing: -2px;
    filter: drop-shadow(2px 2px 4px rgba(0,0,0,0.1));
}

@keyframes gradient {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}

.subtitle {
    color: #4a5568;
    font-size: 24px;
    text-align: center;
    margin-bottom: 40px;
    font-weight: 400;
    letter-spacing: 1px;
}

.glass-card {
    background: rgba(255, 255, 255, 0.7);
    backdrop-filter: blur(20px) saturate(180%);
    border-radius: 20px;
    padding: 35px;
    margin: 25px 0;
    border: 1px solid rgba(255, 255, 255, 0.8);
    box-shadow: 0 20px 40px rgba(0, 0, 0, 0.05);
    transition: all 0.4s cubic-bezier(0.175, 0.885, 0.32, 1.275);
    position: relative;
    overflow: hidden;
}

.glass-card::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg, transparent, rgba(255, 255, 255, 0.1), transparent);
    transition: left 0.5s;
}

.glass-card:hover::before {
    left: 100%;
}

.glass-card:hover {
    transform: translateY(-5px) scale(1.02);
    box-shadow: 0 30px 60px rgba(0, 0, 0, 0.3);
    border-color: rgba(255, 255, 255, 0.3);
}

.section-title {
    color: #2d3748;
    font-size: 36px;
    font-weight: 700;
    margin-bottom: 25px;
    position: relative;
    padding-bottom: 15px;
    letter-spacing: -0.5px;
}

.section-title:after {
    content: '';
    position: absolute;
    left: 0;
    bottom: 0;
    width: 80px;
    height: 5px;
    background: linear-gradient(90deg, #f093fb 0%, #f5576c 100%);
    border-radius: 3px;
    animation: expand 0.5s ease-out;
}

@keyframes expand {
    from { width: 0; }
    to { width: 80px; }
}

.fancy-button {
    background: linear-gradient(45deg, #6c5ce7 0%, #a29bfe 25%, #74b9ff 50%, #0984e3 75%);
    background-size: 200% 200%;
    animation: gradient 3s ease infinite;
    border: none;
    border-radius: 50px;
    color: white;
    font-size: 18px;
    font-weight: 600;
    padding: 18px 45px;
    margin: 10px;
    cursor: pointer;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.15);
    transition: all 0.3s ease;
    text-transform: uppercase;
    letter-spacing: 1px;
    position: relative;
    overflow: hidden;
}

.fancy-button:hover {
    transform: translateY(-3px) scale(1.05);
    box-shadow: 0 15px 40px rgba(0, 0, 0, 0.4);
}

.fancy-button:active {
    transform: translateY(-1px) scale(1.02);
}

.search-box {
    position: relative;
    margin: 30px 0;
}

.search-input {
    width: 100%;
    padding: 25px 30px;
    border: 2px solid transparent;
    border-radius: 60px;
    font-size: 20px;
    background: rgba(255, 255, 255, 0.1);
    backdrop-filter: blur(10px);
    color: white;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.3);
    transition: all 0.3s ease;
}

.search-input::placeholder {
    color: rgba(255, 255, 255, 0.6);
}

.search-input:focus {
    outline: none;
    border-color: #f093fb;
    box-shadow: 0 15px 40px rgba(240, 147, 251, 0.3);
    transform: translateY(-2px);
}

.result-item {
    background: rgba(255, 255, 255, 0.95);
    border-radius: 25px;
    padding: 30px;
    margin: 20px 0;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
    transition: all 0.4s cubic-bezier(0.175, 0.885, 0.32, 1.275);
    border-left: 6px solid;
    border-image: linear-gradient(45deg, #f093fb, #f5576c) 1;
    position: relative;
    overflow: hidden;
}

.result-item::before {
    content: '';
    position: absolute;
    top: -2px;
    left: -2px;
    right: -2px;
    bottom: -2px;
    background: linear-gradient(45deg, #f093fb, #f5576c, #4facfe, #00f2fe);
    border-radius: 25px;
    opacity: 0;
    z-index: -1;
    transition: opacity 0.3s ease;
}

.result-item:hover::before {
    opacity: 1;
}

.result-item:hover {
    transform: translateX(10px) scale(1.02);
    box-shadow: 0 20px 50px rgba(0, 0, 0, 0.3);
}

.result-title {
    color: #302b63;
    font-size: 26px;
    font-weight: 700;
    margin-bottom: 12px;
    transition: color 0.3s ease;
}

.result-item:hover .result-title {
    color: #f093fb;
}

.score-badge {
    display: inline-block;
    background: linear-gradient(45deg, #f093fb 0%, #f5576c 100%);
    color: white;
    padding: 8px 20px;
    border-radius: 25px;
    font-size: 16px;
    font-weight: 600;
    position: absolute;
    top: 30px;
    right: 30px;
    box-shadow: 0 5px 15px rgba(240, 147, 251, 0.4);
}

.stat-card {
    background: linear-gradient(135deg, rgba(240, 147, 251, 0.9) 0%, rgba(245, 87, 108, 0.9) 100%);
    border-radius: 30px;
    padding: 35px;
    margin: 15px;
    box-shadow: 0 15px 40px rgba(0, 0, 0, 0.3);
    text-align: center;
    color: white;
    transition: all 0.4s cubic-bezier(0.175, 0.885, 0.32, 1.275);
    position: relative;
    overflow: hidden;
}

.stat-card:hover {
    transform: translateY(-10px) scale(1.05);
    box-shadow: 0 25px 60px rgba(0, 0, 0, 0.4);
}

.stat-icon {
    font-size: 60px;
    margin-bottom: 20px;
    animation: bounce 2s ease-in-out infinite;
}

@keyframes bounce {
    0%, 100% { transform: translateY(0); }
    50% { transform: translateY(-10px); }
}

.stat-number {
    font-size: 56px;
    font-weight: 900;
    margin-bottom: 10px;
    animation: count 1s ease-out;
}

.progress-bar-container {
    background: rgba(255, 255, 255, 0.2);
    border-radius: 50px;
    padding: 4px;
    margin: 25px 0;
    box-shadow: inset 0 4px 10px rgba(0, 0, 0, 0.3);
}

.progress-bar {
    background: linear-gradient(90deg, #f093fb 0%, #f5576c 30%, #4facfe 60%, #00f2fe 100%);
    background-size: 200% 200%;
    animation: gradient 3s ease infinite;
    height: 35px;
    border-radius: 50px;
    transition: width 0.5s ease;
    display: flex;
    align-items: center;
    justify-content: center;
    color: white;
    font-weight: 700;
    font-size: 18px;
    box-shadow: 0 5px 20px rgba(79, 172, 254, 0.5);
}

.loading-spinner {
    width: 60px;
    height: 60px;
    border: 5px solid rgba(108, 92, 231, 0.2);
    border-radius: 50%;
    border-top-color: #6c5ce7;
    animation: spin 0.8s linear infinite;
    margin: 30px auto;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

.feature-tag {
    display: inline-block;
    background: linear-gradient(45deg, rgba(108, 92, 231, 0.1), rgba(162, 155, 254, 0.1));
    backdrop-filter: blur(10px);
    color: #4a5568;
    padding: 10px 25px;
    border-radius: 30px;
    font-size: 14px;
    font-weight: 500;
    margin: 8px;
    box-shadow: 0 5px 15px rgba(0, 0, 0, 0.05);
    border: 1px solid rgba(108, 92, 231, 0.2);
    animation: float 3s ease-in-out infinite;
    transition: all 0.3s ease;
}

.feature-tag:hover {
    transform: translateY(-5px) scale(1.1);
    box-shadow: 0 10px 25px rgba(0, 0, 0, 0.3);
}

@keyframes float {
    0%, 100% { transform: translateY(0px) rotate(0deg); }
    50% { transform: translateY(-10px) rotate(3deg); }
}

.widget-button {
    background: linear-gradient(45deg, #6c5ce7 0%, #a29bfe 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 15px !important;
    font-size: 18px !important;
    font-weight: 600 !important;
    margin: 10px !important;
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.15) !important;
    transition: all 0.3s ease !important;
    text-transform: uppercase !important;
    letter-spacing: 1px !important;
}

.widget-button:hover {
    transform: translateY(-3px) scale(1.05) !important;
    box-shadow: 0 15px 40px rgba(0, 0, 0, 0.4) !important;
}

.sensor-card {
    background: linear-gradient(135deg, rgba(79, 172, 254, 0.9) 0%, rgba(0, 242, 254, 0.9) 100%);
    border-radius: 25px;
    padding: 30px;
    margin: 15px;
    box-shadow: 0 15px 40px rgba(0, 0, 0, 0.2);
    color: white;
    transition: all 0.4s cubic-bezier(0.175, 0.885, 0.32, 1.275);
}

.sensor-card:hover {
    transform: translateY(-5px) scale(1.02);
    box-shadow: 0 20px 50px rgba(0, 0, 0, 0.3);
}

.sensor-value {
    font-size: 42px;
    font-weight: 900;
    margin: 10px 0;
}

.sensor-label {
    font-size: 18px;
    opacity: 0.9;
    text-transform: uppercase;
    letter-spacing: 1px;
}

.sensor-unit {
    font-size: 20px;
    opacity: 0.8;
}

.real-time-indicator {
    display: inline-block;
    width: 10px;
    height: 10px;
    background: #00ff00;
    border-radius: 50%;
    margin-left: 10px;
    animation: pulse 2s infinite;
}

@keyframes pulse {
    0% { box-shadow: 0 0 0 0 rgba(0, 255, 0, 0.7); }
    70% { box-shadow: 0 0 0 10px rgba(0, 255, 0, 0); }
    100% { box-shadow: 0 0 0 0 rgba(0, 255, 0, 0); }
}

.mqtt-status {
    position: fixed;
    top: 20px;
    right: 20px;
    background: rgba(255, 255, 255, 0.9);
    padding: 15px 25px;
    border-radius: 30px;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
    z-index: 1000;
}

.control-panel {
    background: rgba(255, 255, 255, 0.9);
    border-radius: 20px;
    padding: 25px;
    margin: 20px 0;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.15);
}

.control-item {
    margin: 15px 0;
    padding: 15px;
    border-radius: 15px;
    background: rgba(108, 92, 231, 0.05);
    transition: all 0.3s ease;
}

.control-item:hover {
    background: rgba(108, 92, 231, 0.1);
    transform: translateX(5px);
}

.sensor-toggle {
    background: #e0e0e0;
    border-radius: 30px;
    width: 60px;
    height: 30px;
    position: relative;
    cursor: pointer;
    transition: background 0.3s ease;
}

.sensor-toggle.active {
    background: #6c5ce7;
}

.sensor-toggle-handle {
    position: absolute;
    top: 3px;
    left: 3px;
    width: 24px;
    height: 24px;
    background: white;
    border-radius: 50%;
    transition: left 0.3s ease;
}

.sensor-toggle.active .sensor-toggle-handle {
    left: 33px;
}

.help-modal {
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: rgba(0, 0, 0, 0.7);
    display: flex;
    justify-content: center;
    align-items: center;
    z-index: 9999;
    animation: fadeIn 0.3s ease-out;
}

.help-content {
    background: white;
    max-width: 800px;
    width: 90%;
    max-height: 80vh;
    overflow-y: auto;
    border-radius: 20px;
    padding: 40px;
    box-shadow: 0 25px 50px rgba(0, 0, 0, 0.3);
    position: relative;
}

.help-close {
    position: absolute;
    top: 20px;
    right: 20px;
    font-size: 30px;
    color: #4a5568;
    cursor: pointer;
    transition: all 0.3s ease;
}

.help-close:hover {
    color: #f5576c;
    transform: rotate(90deg);
}

.help-title {
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 20px;
    color: #2d3748;
}

.help-section {
    margin-bottom: 25px;
}

.help-section h3 {
    font-size: 22px;
    color: #4a5568;
    margin-bottom: 15px;
}

.help-section p {
    font-size: 16px;
    line-height: 1.6;
    color: #718096;
    margin-bottom: 15px;
}

.help-section ul {
    margin-left: 20px;
    margin-bottom: 15px;
}

.help-section li {
    font-size: 16px;
    line-height: 1.6;
    color: #718096;
    margin-bottom: 8px;
}

.guide-btn {
    background: linear-gradient(45deg, #00cec9 0%, #4facfe 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 50px !important;
    font-size: 16px !important;
    font-weight: 600 !important;
    margin: 10px !important;
    padding: 10px 20px !important;
    box-shadow: 0 8px 15px rgba(0, 0, 0, 0.1) !important;
    transition: all 0.3s ease !important;
}

.guide-btn:hover {
    transform: translateY(-3px) scale(1.05) !important;
    box-shadow: 0 12px 25px rgba(0, 0, 0, 0.2) !important;
}

@keyframes fadeIn {
    from { opacity: 0; }
    to { opacity: 1; }
}
</style>
"""

In [ ]:
# @title SimpleStemmer Class

class SimpleStemmer:
    """Enhanced simple stemmer without external dependencies"""
    def __init__(self):
        self.rules = [
            ('sses', 'ss'),
            ('ies', 'i'),
            ('ss', 'ss'),
            ('s', ''),
            ('eed', 'ee'),
            ('ed', ''),
            ('ing', ''),
            ('ation', 'ate'),
            ('ator', 'ate'),
            ('alism', 'al'),
            ('iveness', 'ive'),
            ('fulness', 'ful'),
            ('ousness', 'ous'),
            ('ality', 'al'),
            ('ivity', 'ive'),
            ('ability', 'able'),
            ('ibility', 'ible')
        ]

    def stem(self, word):
        """Apply enhanced stemming rules"""
        word = word.lower().strip()

        # Apply suffix rules
        for suffix, replacement in self.rules:
            if word.endswith(suffix):
                return word[:-len(suffix)] + replacement

        return word

In [ ]:
# @title SensorDataManager
class SensorDataManager:
    def __init__(self):
        self.use_fake_data = True  # Will use simulated data if Firebase data not available
        self.data = {
            "indoor": [],
            "outdoor": []
        }
        self.history = {
            "indoor": [],
            "outdoor": []
        }
        self.max_data_points = 100  # Default maximum data points to retain
        self.refresh_interval = 5  # Default refresh interval in seconds

        # Load initial data
        self.load_data()

    def load_data(self):
        """Attempts to load data from Firebase, falls back to simulated data if unavailable"""
        # Try loading from Firebase first
        firebase_data = self.load_firebase_data()

        # If Firebase data loaded successfully, use it
        if firebase_data:
            self.use_fake_data = False
            self.data = firebase_data["latest"]
            self.history = firebase_data["history"]
        else:
            # Otherwise generate simulated data
            self.use_fake_data = True
            self.generate_fake_data()

    def load_firebase_data(self):
        """Attempt to load data from Firebase"""
        try:
            # In a real application, this would connect to Firebase
            # For this example, let's simulate firebase data based on the provided structure

            # Current timestamp
            now = datetime.now().isoformat()

            # Example data matching the database structure we've been given
            firebase_data = {
                "latest": {
                    "indoor": {
                        "Temperature": 28.66,
                        "Humidity": 48.64,
                        "Pressure": 977.71,
                        "timestamp": now
                    },
                    "outdoor": {
                        "Temperature": 31.91,
                        "Humidity": 45.87,
                        "Day Light": 1024,  # Fixed key from 'DLIGHT' to 'Day Light'
                        "timestamp": now
                    }
                },
                "history": {
                    "indoor": [],
                    "outdoor": []
                }
            }

            # Generate some historical data
            for i in range(24):
                # Indoor history
                time_offset = timedelta(hours=i)
                timestamp = (datetime.now() - time_offset).isoformat()

                indoor_entry = {
                    "Temperature": 28.66 + random.uniform(-3, 3),
                    "Humidity": 48.64 + random.uniform(-5, 5),
                    "Pressure": 977.71 + random.uniform(-10, 10),
                    "timestamp": timestamp
                }
                firebase_data["history"]["indoor"].append(indoor_entry)

                # Outdoor history
                outdoor_entry = {
                    "Temperature": 31.91 + random.uniform(-4, 4),
                    "Humidity": 45.87 + random.uniform(-7, 7),
                    "Day Light": max(0, 1024 + random.uniform(-500, 200)),  # Fixed key
                    "timestamp": timestamp
                }
                firebase_data["history"]["outdoor"].append(outdoor_entry)

            # Sort by timestamp
            firebase_data["history"]["indoor"].sort(key=lambda x: x["timestamp"])
            firebase_data["history"]["outdoor"].sort(key=lambda x: x["timestamp"])

            return firebase_data

        except Exception as e:
            print(f"Error loading Firebase data: {e}")
            return None

    def generate_fake_data(self):
        """Generate simulated sensor data"""
        # Current timestamp
        now = datetime.now().isoformat()

        # Generate latest readings
        self.data["indoor"] = {
            "Temperature": 24.5 + random.uniform(-1, 1),
            "Humidity": 45.0 + random.uniform(-5, 5),
            "Pressure": 975.0 + random.uniform(-5, 5),
            "Distance": 150.0 + random.uniform(-10, 10),
            "timestamp": now
        }

        self.data["outdoor"] = {
            "Temperature": 28.0 + random.uniform(-3, 3),
            "Humidity": 35.0 + random.uniform(-7, 7),
            "Pressure": 965.0 + random.uniform(-10, 10),
            "Day Light": random.uniform(800, 1200),  # Fixed key
            "timestamp": now
        }

        # Generate history data if empty
        if not self.history["indoor"]:
            for i in range(24):
                time_offset = timedelta(hours=i)
                timestamp = (datetime.now() - time_offset).isoformat()

                # Generate indoor history
                indoor_entry = {
                    "Temperature": 24.5 + random.uniform(-3, 3) + i * 0.1,
                    "Humidity": 45.0 + random.uniform(-8, 8),
                    "Pressure": 975.0 + random.uniform(-10, 10),
                    "Distance": 150.0 + random.uniform(-20, 20),
                    "timestamp": timestamp
                }
                self.history["indoor"].append(indoor_entry)

                # Generate outdoor history
                outdoor_entry = {
                    "Temperature": 28.0 + random.uniform(-5, 5) + i * 0.15,
                    "Humidity": 35.0 + random.uniform(-10, 10),
                    "Pressure": 965.0 + random.uniform(-15, 15),
                    "Day Light": max(0, 1000 - i * 40 + random.uniform(-100, 100)),  # Fixed key
                    "timestamp": timestamp
                }
                self.history["outdoor"].append(outdoor_entry)

        # Sort by timestamp
        self.history["indoor"].sort(key=lambda x: x["timestamp"])
        self.history["outdoor"].sort(key=lambda x: x["timestamp"])

    def get_latest_data(self):
        """Get latest sensor readings"""
        # Update data if using fake data
        if self.use_fake_data:
            self.generate_fake_data()

        return self.data

    def get_history_data(self, num_points=None):
        """Get historical sensor data

        Args:
            num_points: Number of most recent data points to return. If None, return all available.

        Returns:
            Dictionary containing indoor and outdoor history data.
        """
        # If no specific number requested, use the max data points setting
        if num_points is None:
            num_points = self.max_data_points

        result = {
            "indoor": self.history["indoor"][-num_points:] if self.history["indoor"] else [],
            "outdoor": self.history["outdoor"][-num_points:] if self.history["outdoor"] else []
        }

        return result

    def get_data_status(self):
        """Get status information about the sensor data"""
        now = datetime.now().isoformat()

        # Get latest timestamp from data
        indoor_timestamp = self.data["indoor"].get("timestamp", "") if self.data["indoor"] else ""
        outdoor_timestamp = self.data["outdoor"].get("timestamp", "") if self.data["outdoor"] else ""

        # Format the display timestamp
        if indoor_timestamp and 'T' in indoor_timestamp:
            display_timestamp = indoor_timestamp.replace('T', ' ').split('.')[0]
        elif outdoor_timestamp and 'T' in outdoor_timestamp:
            display_timestamp = outdoor_timestamp.replace('T', ' ').split('.')[0]
        else:
            display_timestamp = now.replace('T', ' ').split('.')[0]

        return {
            "data_loaded": not self.use_fake_data,
            "last_updated": display_timestamp,
            "points_available": {
                "indoor": len(self.history["indoor"]),
                "outdoor": len(self.history["outdoor"])
            }
        }

    def set_refresh_interval(self, seconds):
        """Set the data refresh interval in seconds"""
        self.refresh_interval = max(1, seconds)  # Ensure at least 1 second

    def set_max_data_points(self, points):
        """Set the maximum number of data points to keep in history"""
        self.max_data_points = max(5, points)  # Ensure at least 5 data points

    def export_data(self):
        """Export all available data for download"""
        return self.history

In [ ]:
# @title MQTTSearchEngine Class

class MQTTSearchEngine:
    def __init__(self):
        """Initialize the enhanced search engine for mqtt.org with Firebase"""
        self.base_url = "https://mqtt.org"
        self.pages = []
        self.word_locations = defaultdict(list)
        self.stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on',
                          'at', 'to', 'for', 'of', 'with', 'by', 'is',
                          'are', 'was', 'were', 'be', 'been', 'has',
                          'have', 'had', 'do', 'does', 'did'}
        self.stemmer = SimpleStemmer()
        self.search_history = []
        self.crawl_times = []
        self.url_to_id = {}  # Map URLs to numeric IDs
        self.id_to_url = {}  # Map numeric IDs to URLs
        self.firebase_url = FIREBASE_URL
        self.sensor_manager = SensorDataManager()

        self.english_words = english_words

    def is_valid_word(self, word):
        """Check if a word is valid English"""
        # Allow numbers
        if word.isdigit():
            return True

        # Check if it's in English dictionary
        if word.lower() in english_words:
            return True

        # Check WordNet
        if wordnet.synsets(word):
            return True

        # Allow common technical terms
        technical_terms = {'mqtt', 'iot', 'http', 'tcp', 'ssl', 'json', 'api', 'qos'}
        if word.lower() in technical_terms:
            return True

        return False

    def process_word(self, word):
        """Process a word using enhanced stemming"""
        return self.stemmer.stem(word.lower())

    def fetch_page(self, url):
        """Fetch a single page from the given URL"""
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                # Ensure proper encoding
                response.encoding = response.apparent_encoding or 'utf-8'
                return response.text
            else:
                print(f"Failed to fetch {url}: Status code {response.status_code}")
                return None
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return None

    def extract_text_from_html(self, html_content):
        """Extract clean text from HTML content"""
        soup = BeautifulSoup(html_content, 'html.parser', from_encoding='utf-8')

        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()

        # Get text
        text = soup.get_text()

        # Clean up text - remove non-ASCII characters
        text = text.encode('ascii', 'ignore').decode('ascii')

        # Clean up whitespace
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)

        return text

    def find_links(self, url, html_content):
        """Find all links on a page that belong to mqtt.org"""
        soup = BeautifulSoup(html_content, 'html.parser')
        links = set()

        for link in soup.find_all('a', href=True):
            href = link['href']

            # Convert relative URLs to absolute
            if href.startswith('/'):
                href = self.base_url + href

            # Only include mqtt.org links
            if href.startswith(self.base_url):
                # Remove anchors and query parameters for simplicity
                href = href.split('#')[0].split('?')[0]
                links.add(href)

        return links

    def crawl_site(self, max_pages=20, progress_callback=None):
        """Crawl the mqtt.org website and collect pages"""
        start_time = time.time()
        print("Starting to crawl mqtt.org...")

        visited = set()
        to_visit = {self.base_url}
        page_id = 0

        while to_visit and len(visited) < max_pages:
            url = to_visit.pop()

            if url in visited:
                continue

            print(f"Crawling page {page_id + 1}/{max_pages}: {url}")

            # Progress callback for UI
            if progress_callback:
                progress_callback(page_id + 1, max_pages, url)

            # Fetch the page
            html_content = self.fetch_page(url)
            if not html_content:
                continue

            # Extract text
            text = self.extract_text_from_html(html_content)

            # Extract title from HTML
            soup = BeautifulSoup(html_content, 'html.parser')
            title_tag = soup.find('title')
            title = title_tag.text if title_tag else url.split('/')[-1] or 'Home'

            # Store page information with numeric ID
            page_data = {
                'id': page_id,
                'url': url,
                'title': title,
                'content': text,
                'crawl_time': datetime.now().isoformat()
            }
            self.pages.append(page_data)

            # Save page to Firebase immediately
            self.save_page_to_firebase(page_data)

            # Map URL to ID and vice versa
            self.url_to_id[url] = page_id
            self.id_to_url[page_id] = url

            # Mark as visited
            visited.add(url)
            page_id += 1

            # Find new links
            new_links = self.find_links(url, html_content)
            to_visit.update(new_links - visited)

            # Be polite to the server
            time.sleep(0.5)

        end_time = time.time()
        self.crawl_times.append(end_time - start_time)
        print(f"Crawling complete. Found {len(self.pages)} pages in {end_time - start_time:.2f} seconds.")

    def save_page_to_firebase(self, page_data):
        """Save a single page to Firebase"""
        try:
            # Save page data
            page_url = f"{self.firebase_url}pages/{page_data['id']}.json"
            response = requests.put(page_url, json=page_data)
            if response.status_code != 200:
                print(f"Failed to save page {page_data['id']} to Firebase")
        except Exception as e:
            print(f"Error saving page to Firebase: {str(e)}")

    def build_index(self):
        """Build the word index from crawled pages"""
        print("Building index with enhanced stemming...")
        self.word_locations.clear()

        # Load pages from Firebase if not in memory
        if not self.pages:
            self.load_pages_from_firebase()

        for page in self.pages:
            # Get all words from content - only ASCII alphanumeric
            words = re.findall(r'\b[a-zA-Z0-9]+\b', page['content'])

            # Count word frequencies
            word_counts = defaultdict(int)
            for word in words:
                # Process word with stemming
                processed_word = self.process_word(word)

                # Only index valid words
                if (processed_word not in self.stop_words and
                    len(processed_word) > 2 and
                    processed_word.isalnum() and
                    self.is_valid_word(processed_word)):  # Ensure alphanumeric only
                    word_counts[processed_word] += 1

            # Add to index
            for word, count in word_counts.items():
                self.word_locations[word].append((page['id'], count))

        print(f"Index built with {len(self.word_locations)} unique words using enhanced stemming")

    def clean_existing_index(self):
        """Clean the existing index to remove corrupted entries"""
        cleaned_locations = defaultdict(list)

        for word, locations in self.word_locations.items():
            # Only keep ASCII alphanumeric words
            if word.isalnum() and all(ord(c) < 128 for c in word):
                cleaned_locations[word] = locations

        self.word_locations = cleaned_locations
        print(f"Cleaned index: {len(self.word_locations)} valid terms remaining")

    def save_index_to_firebase(self):
        """Save the index to Firebase with numeric IDs"""
        index_data = {}

        # Clean index before saving
        self.clean_existing_index()

        # Prepare index data
        for term, locations in self.word_locations.items():
            # Create numeric DocIDs list for each term
            doc_ids = []
            for page_id, frequency in locations:
                doc_ids.append(page_id)  # Use numeric ID instead of URL

            # Required structure: term and DocIDs
            index_data[term] = {
                'term': term,
                'DocIDs': doc_ids
            }

        # Save main index to Firebase
        try:
            index_url = f"{self.firebase_url}index.json"
            response = requests.put(index_url, json=index_data)
            if response.status_code == 200:
                print(f"Index saved to Firebase successfully")
            else:
                print(f"Failed to save index to Firebase: {response.status_code}")

            # Convert integer keys to strings for Firebase
            # Firebase doesn't handle integer keys well in JSON
            url_to_id_str = {}
            for url, id_val in self.url_to_id.items():
                # Clean URL for Firebase key compliance
                # Replace problematic characters with underscores
                clean_url = url.replace('/', '_').replace('.', '_').replace(':', '_')
                url_to_id_str[clean_url] = {'url': url, 'id': id_val}

            id_to_url_str = {}
            for id_val, url in self.id_to_url.items():
                # Convert integer ID to string
                id_to_url_str[str(id_val)] = url

            # Save URL mapping with string keys
            mapping_data = {
                'url_to_id': url_to_id_str,
                'id_to_url': id_to_url_str
            }

            mapping_url = f"{self.firebase_url}mapping.json"
            response = requests.put(mapping_url, json=mapping_data)
            if response.status_code == 200:
                print(f"URL mapping saved to Firebase successfully")
            else:
                print(f"Failed to save mapping to Firebase: {response.status_code}")
                # Try to get more error info
                print(f"Error response: {response.text}")

        except Exception as e:
            print(f"Error saving to Firebase: {str(e)}")

    def load_index_from_firebase(self):
        """Load the index from Firebase"""
        try:
            # Load index
            index_url = f"{self.firebase_url}index.json"
            response = requests.get(index_url)
            if response.status_code == 200:
                index_data = response.json()
                if index_data:
                    self.word_locations.clear()
                    for term, data in index_data.items():
                        if isinstance(data, dict) and 'DocIDs' in data:
                            # Reconstruct word locations with frequency 1 for simplicity
                            for doc_id in data['DocIDs']:
                                self.word_locations[term].append((doc_id, 1))
                    print(f"Index loaded from Firebase: {len(self.word_locations)} terms")
                else:
                    print("No index found in Firebase")

            # Load mapping
            mapping_url = f"{self.firebase_url}mapping.json"
            response = requests.get(mapping_url)
            if response.status_code == 200:
                mapping_data = response.json()
                if mapping_data:
                    # Load url_to_id with URL restoration
                    self.url_to_id = {}
                    url_to_id_data = mapping_data.get('url_to_id', {})
                    for clean_url, url_info in url_to_id_data.items():
                        if isinstance(url_info, dict):
                            original_url = url_info.get('url', '')
                            id_val = url_info.get('id', 0)
                            self.url_to_id[original_url] = id_val

                    # Load id_to_url with string key conversion
                    self.id_to_url = {}
                    id_to_url_data = mapping_data.get('id_to_url', {})
                    for str_id, url in id_to_url_data.items():
                        self.id_to_url[int(str_id)] = url

                    print(f"URL mapping loaded from Firebase")

        except Exception as e:
            print(f"Error loading from Firebase: {str(e)}")

    def load_pages_from_firebase(self):
        """Load all pages from Firebase"""
        try:
            pages_url = f"{self.firebase_url}pages.json"
            response = requests.get(pages_url)
            if response.status_code == 200:
                pages_data = response.json()
                if pages_data:
                    self.pages = []
                    for page_id, page_data in pages_data.items():
                        if page_data:
                            self.pages.append(page_data)
                    self.pages.sort(key=lambda x: x['id'])
                    print(f"Loaded {len(self.pages)} pages from Firebase")
                else:
                    print("No pages found in Firebase")
        except Exception as e:
            print(f"Error loading pages from Firebase: {str(e)}")

    def search(self, query):
        """Enhanced search for a query in the index"""
        self.search_history.append({'query': query, 'time': datetime.now()})

        # Load index from Firebase if not in memory
        if not self.word_locations:
            self.load_index_from_firebase()

        # Load pages from Firebase if not in memory
        if not self.pages:
            self.load_pages_from_firebase()

        results = []
        query_words = re.findall(r'\w+', query.lower())

        # Process query words with stemming
        processed_query_words = []
        for word in query_words:
            processed_word = self.process_word(word)
            if processed_word not in self.stop_words and len(processed_word) > 2:
                processed_query_words.append(processed_word)

        # Find pages containing query words
        page_scores = defaultdict(int)

        for query_word in processed_query_words:
            if query_word in self.word_locations:
                for page_id, frequency in self.word_locations[query_word]:
                    page_scores[page_id] += frequency

        # Sort by relevance score
        sorted_pages = sorted(page_scores.items(), key=lambda x: x[1], reverse=True)

        # Return top results
        for page_id, score in sorted_pages[:10]:  # Top 10 results
            # Find page in self.pages
            page = next((p for p in self.pages if p['id'] == page_id), None)
            if page:
                results.append({
                    'id': page_id,
                    'title': page['title'],
                    'url': page['url'],
                    'score': score,
                    'snippet': page['content'][:200] + '...'
                })

        return results

In [ ]:
# @title Helper Functions and Initialization

def show_premium_status(message, status_type='success'):
    """Display premium status message with enhanced animations"""
    icons = {
        'success': '✨',
        'error': '❌',
        'warning': '⚠️',
        'info': 'ℹ️',
        'loading': '⏳'
    }

    colors = {
        'success': 'linear-gradient(135deg, #11998e 0%, #38ef7d 100%)',
        'error': 'linear-gradient(135deg, #eb3349 0%, #f45c43 100%)',
        'warning': 'linear-gradient(135deg, #f2994a 0%, #f2c94c 100%)',
        'info': 'linear-gradient(135deg, #4facfe 0%, #00f2fe 100%)',
        'loading': 'linear-gradient(135deg, #f093fb 0%, #f5576c 100%)'
    }

    display(HTML(f'''
    <div style="background: {colors[status_type]}; color: white; padding: 20px;
                border-radius: 15px; margin: 10px 0; box-shadow: 0 8px 20px rgba(0,0,0,0.2);
                animation: fadeIn 0.5s ease-out;">
        <span style="font-size: 24px; margin-right: 10px;">{icons[status_type]}</span>
        <span style="font-size: 18px; font-weight: 600;">{message}</span>
    </div>
    '''))

def create_fancy_progress_bar(current, total, message=""):
    """Create enhanced progress bar with gradient animation"""
    percentage = (current / total) * 100
    return HTML(f'''
    <div class="glass-card">
        <p style="color: #2d3748; font-size: 20px; margin-bottom: 15px; text-align: center;">{message}</p>
        <div class="progress-bar-container">
            <div class="progress-bar" style="width: {percentage}%;">
                {int(percentage)}%
            </div>
        </div>
        <p style="color: #718096; text-align: center; font-size: 18px;">
            {current} out of {total} pages crawled
        </p>
    </div>
    ''')

# Initialize the search engine
engine = MQTTSearchEngine()
sensor_manager = engine.sensor_manager

In [ ]:
# @title Guide Helpers

# Guide content for each screen
def main_menu_guide():
    return """
    <div class="help-modal">
        <div class="help-content">
            <div class="help-close" onclick="this.parentElement.parentElement.style.display='none';">×</div>
            <h2 class="help-title">📚 Main Menu Guide</h2>

            <div class="help-section">
                <h3>Welcome to the MQTT Search Engine</h3>
                <p>This application combines search functionality with sensor data visualization. Use the buttons below to navigate to different screens:</p>
                <ul>
                    <li><strong>Admin Panel</strong> - Manage the search engine: crawl websites, build and save the search index</li>
                    <li><strong>Smart Search</strong> - Search through indexed content from mqtt.org</li>
                    <li><strong>Search Analytics</strong> - View statistics and visualizations about the search index</li>
                    <li><strong>Sensor Readings</strong> - View the latest sensor data from indoor and outdoor environments</li>
                    <li><strong>Sensor Analytics</strong> - Analyze and visualize sensor data patterns over time</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Getting Started</h3>
                <p>To get the most out of this application:</p>
                <ol>
                    <li>First visit the <strong>Admin Panel</strong> to build the search index (if not already done)</li>
                    <li>Use <strong>Smart Search</strong> to find information about MQTT</li>
                    <li>Check out <strong>Sensor Readings</strong> to view data from environmental sensors</li>
                    <li>Explore <strong>Sensor Analytics</strong> for in-depth analysis of sensor data trends</li>
                </ol>
            </div>
        </div>
    </div>
    """

def admin_screen_guide():
    return """
    <div class="help-modal">
        <div class="help-content">
            <div class="help-close" onclick="this.parentElement.parentElement.style.display='none';">×</div>
            <h2 class="help-title">📚 Admin Panel Guide</h2>

            <div class="help-section">
                <h3>Managing the Search Engine</h3>
                <p>This panel allows you to control the search engine functionality:</p>
                <ul>
                    <li><strong>Pages to crawl</strong> - Set how many pages from mqtt.org to index</li>
                    <li><strong>Start Crawling</strong> - Begin fetching and processing pages from mqtt.org</li>
                    <li><strong>Build Index</strong> - Create a searchable index from the crawled pages</li>
                    <li><strong>Save to Firebase</strong> - Store the index in the cloud database</li>
                    <li><strong>Load from Firebase</strong> - Retrieve previously saved index</li>
                    <li><strong>Clean Index</strong> - Remove any corrupted entries from the index</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Recommended Workflow</h3>
                <ol>
                    <li>Set the number of pages you want to crawl (5-20 is recommended)</li>
                    <li>Click <strong>Start Crawling</strong> and wait for the process to complete</li>
                    <li>Click <strong>Build Index</strong> to process the crawled pages</li>
                    <li>Click <strong>Save to Firebase</strong> to store your index for future use</li>
                </ol>
                <p>If you've previously saved an index, you can load it using <strong>Load from Firebase</strong> instead of crawling again.</p>
            </div>
        </div>
    </div>
    """

def search_screen_guide():
    return """
    <div class="help-modal">
        <div class="help-content">
            <div class="help-close" onclick="this.parentElement.parentElement.style.display='none';">×</div>
            <h2 class="help-title">📚 Smart Search Guide</h2>

            <div class="help-section">
                <h3>Searching the MQTT Index</h3>
                <p>Use this screen to search through the indexed content from mqtt.org:</p>
                <ul>
                    <li>Enter your search query in the search box</li>
                    <li>Click <strong>Search</strong> to find relevant results</li>
                    <li>Results are displayed below with relevance scores</li>
                    <li>Click on any result link to visit the original page</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Search Tips</h3>
                <ul>
                    <li>Use specific terms related to MQTT for best results</li>
                    <li>Try different variations of terms if you don't find what you need</li>
                    <li>Results are ranked by relevance score - higher scores are more relevant</li>
                    <li>Remember that only mqtt.org pages are indexed in this demo</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Example Queries</h3>
                <ul>
                    <li>"broker" - Find information about MQTT brokers</li>
                    <li>"protocol security" - Learn about security aspects of MQTT</li>
                    <li>"publish subscribe" - Discover details about the pub/sub pattern</li>
                </ul>
            </div>
        </div>
    </div>
    """

def sensors_screen_guide():
    return """
    <div class="help-modal">
        <div class="help-content">
            <div class="help-close" onclick="this.parentElement.parentElement.style.display='none';">×</div>
            <h2 class="help-title">📚 Sensor Readings Guide</h2>

            <div class="help-section">
                <h3>Viewing Sensor Data</h3>
                <p>This screen displays stored readings from indoor and outdoor environmental sensors:</p>
                <ul>
                    <li><strong>Indoor sensors</strong> measure temperature, humidity, pressure, and distance/motion</li>
                    <li><strong>Outdoor sensors</strong> measure temperature, humidity, pressure, and light intensity</li>
                    <li>Data is pre-collected and stored in Firebase or simulated if no data is available</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Display Controls</h3>
                <ul>
                    <li><strong>Display Refresh</strong> - How often to update the display (in seconds)</li>
                    <li><strong>Data Points to Display</strong> - How many historical data points to include</li>
                    <li><strong>Show Indoor/Outdoor</strong> - Toggle which sensor groups to display</li>
                    <li><strong>Refresh</strong> - Manually update the display</li>
                    <li><strong>Export Data</strong> - Export sensor data for external use</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Understanding the Data</h3>
                <ul>
                    <li><strong>Temperature</strong> is measured in °C (Celsius)</li>
                    <li><strong>Humidity</strong> is measured in % (percentage)</li>
                    <li><strong>Pressure</strong> is measured in Pa (Pascal)</li>
                    <li><strong>Distance</strong> is measured in mm (millimeters)</li>
                    <li><strong>Light</strong> is measured in Lux (luminous flux per unit area)</li>
                </ul>
            </div>
        </div>
    </div>
    """

def sensor_analytics_guide():
    return """
    <div class="help-modal">
        <div class="help-content">
            <div class="help-close" onclick="this.parentElement.parentElement.style.display='none';">×</div>
            <h2 class="help-title">📚 Sensor Analytics Guide</h2>

            <div class="help-section">
                <h3>Analyzing Sensor Data</h3>
                <p>This screen provides advanced analysis and visualization of sensor data:</p>
                <ul>
                    <li>View key statistics about indoor and outdoor environments</li>
                    <li>Explore interactive charts showing data patterns over time</li>
                    <li>Compare indoor vs outdoor conditions</li>
                    <li>See statistical summaries of all sensor readings</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Available Visualizations</h3>
                <ul>
                    <li><strong>Temperature Trends</strong> - Compare indoor and outdoor temperature patterns</li>
                    <li><strong>Humidity Comparison</strong> - See differences in humidity levels</li>
                    <li><strong>Pressure Trends</strong> - Track atmospheric pressure changes</li>
                    <li><strong>Light Intensity</strong> - Monitor outdoor light levels over time</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Statistical Analysis</h3>
                <p>For each sensor measurement, you can view:</p>
                <ul>
                    <li><strong>Mean</strong> - Average value</li>
                    <li><strong>Standard Deviation</strong> - Measure of variation</li>
                    <li><strong>Minimum</strong> - Lowest recorded value</li>
                    <li><strong>Maximum</strong> - Highest recorded value</li>
                </ul>
                <p>Use these statistics to identify patterns, anomalies, or significant changes in environmental conditions.</p>
            </div>
        </div>
    </div>
    """

def stats_screen_guide():
    return """
    <div class="help-modal">
        <div class="help-content">
            <div class="help-close" onclick="this.parentElement.parentElement.style.display='none';">×</div>
            <h2 class="help-title">📚 Search Analytics Guide</h2>

            <div class="help-section">
                <h3>Understanding Search Engine Performance</h3>
                <p>This screen provides insights into the search engine's performance and content:</p>
                <ul>
                    <li>View statistics about indexed pages and terms</li>
                    <li>Explore visualizations of index content</li>
                    <li>Track search history and patterns</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Key Metrics</h3>
                <ul>
                    <li><strong>Pages in Firebase</strong> - Total number of crawled and stored pages</li>
                    <li><strong>Clean Terms</strong> - Number of unique terms in the search index</li>
                    <li><strong>Total Searches</strong> - How many searches have been performed</li>
                    <li><strong>Total Crawl Time</strong> - Cumulative time spent crawling websites</li>
                </ul>
            </div>

            <div class="help-section">
                <h3>Available Visualizations</h3>
                <ul>
                    <li><strong>Word Frequency</strong> - Most common terms in the index</li>
                    <li><strong>Word Length Distribution</strong> - Distribution of term lengths</li>
                    <li><strong>Firebase Storage</strong> - Breakdown of storage usage</li>
                    <li><strong>Recent Search History</strong> - Log of recent search queries</li>
                    <li><strong>Document Coverage</strong> - Pages with most indexed terms</li>
                    <li><strong>Index Growth</strong> - How the index expanded over time</li>
                </ul>
                <p>These visualizations help you understand the content and structure of the search index.</p>
            </div>
        </div>
    </div>
    """

In [ ]:
# @title Main Menu Screen

def main_menu():
    clear_output()
    display(HTML(premium_css))

    display(HTML('''
    <div class="app-container">
        <h1 class="main-title">MQTT Search Engine</h1>
        <p class="subtitle">Firebase-Powered Search Platform with Pre-Collected Sensor Data</p>

        <div class="glass-card" style="margin-top: 40px;">
            <h3 style="color: #2d3748; margin-bottom: 20px; font-size: 28px;">🚀 Premium Features v4.0</h3>
            <div style="display: flex; flex-wrap: wrap; justify-content: center;">
                <span class="feature-tag">Firebase Database</span>
                <span class="feature-tag">Pre-Collected Sensor Data</span>
                <span class="feature-tag">Enhanced Displays</span>
                <span class="feature-tag">Clean ASCII Index</span>
                <span class="feature-tag">Fixed Encoding Issues</span>
                <span class="feature-tag">Export Capabilities</span>
                <span class="feature-tag">Interactive Visualizations</span>
                <span class="feature-tag">Separate Analytics</span>
            </div>
        </div>
    </div>
    '''))

    # Show data status
    data_status = sensor_manager.get_data_status()
    if data_status["data_loaded"]:
        data_source = "Stored" if not sensor_manager.use_fake_data else "Simulated"
        data_info = f"Using {data_source} Data: {data_status['points_available']['indoor']} indoor points"
        data_color = "#4facfe"
    else:
        data_info = "No data available"
        data_color = "#f5576c"

    display(HTML(f'''
    <div class="mqtt-status">
        <span style="display: inline-block; width: 10px; height: 10px; background: {data_color};
                    border-radius: 50%; margin-right: 10px;"></span>
        {data_info}
    </div>
    '''))

    # Create interactive buttons
    admin_btn = widgets.Button(
        description="⚙️ Admin Panel",
        style={'button_color': '#f093fb'},
        layout=widgets.Layout(width='200px', height='60px')
    )
    admin_btn.add_class('widget-button')

    search_btn = widgets.Button(
        description="🔍 Smart Search",
        style={'button_color': '#f5576c'},
        layout=widgets.Layout(width='200px', height='60px')
    )
    search_btn.add_class('widget-button')

    stats_btn = widgets.Button(
        description="📊 Search Analytics",
        style={'button_color': '#4facfe'},
        layout=widgets.Layout(width='200px', height='60px')
    )
    stats_btn.add_class('widget-button')

    sensors_btn = widgets.Button(
        description="🌡️ Sensor Readings",
        style={'button_color': '#00f2fe'},
        layout=widgets.Layout(width='200px', height='60px')
    )
    sensors_btn.add_class('widget-button')

    sensor_analytics_btn = widgets.Button(
        description="📈 Sensor Analytics",
        style={'button_color': '#43e97b'},
        layout=widgets.Layout(width='200px', height='60px')
    )
    sensor_analytics_btn.add_class('widget-button')

    # Add guide button
    guide_btn = widgets.Button(
        description="❓ Help Guide",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='150px', height='50px')
    )
    guide_btn.add_class('guide-btn')

    # Connect functions to buttons
    admin_btn.on_click(lambda b: admin_screen())
    search_btn.on_click(lambda b: search_screen())
    stats_btn.on_click(lambda b: stats_screen())
    sensors_btn.on_click(lambda b: sensors_screen())
    sensor_analytics_btn.on_click(lambda b: sensor_analytics_screen())
    guide_btn.on_click(lambda b: display(HTML(main_menu_guide())))

    # Display buttons with enhanced styling
    display(HTML('<div class="glass-card" style="text-align: center;">'))
    display(widgets.HBox([admin_btn, search_btn, stats_btn],
                    layout=widgets.Layout(justify_content='center', gap='20px')))
    display(widgets.HBox([sensors_btn, sensor_analytics_btn],
                    layout=widgets.Layout(justify_content='center', gap='20px')))
    display(widgets.HBox([guide_btn],
                    layout=widgets.Layout(justify_content='center', margin='20px 0 0 0')))
    display(HTML('</div>'))

In [ ]:
# @title Admin Screen

def admin_screen():
    clear_output()
    display(HTML(premium_css))

    display(HTML('''
    <div class="app-container">
        <h1 class="main-title">⚙️ Admin Dashboard</h1>
        <p class="subtitle">Firebase-Powered Search Engine Management</p>
    </div>
    '''))

    # Enhanced slider for pages
    pages_slider = widgets.IntSlider(
        value=20,
        min=5,
        max=100,
        step=5,
        description='Pages to crawl:',
        style={'description_width': 'initial', 'handle_color': '#f093fb'}
    )

    # Enhanced buttons
    crawl_btn = widgets.Button(
        description="🌐 Start Crawling",
        style={'button_color': '#f093fb'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    index_btn = widgets.Button(
        description="🔨 Build Index",
        style={'button_color': '#f5576c'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    save_btn = widgets.Button(
        description="☁️ Save to Firebase",
        style={'button_color': '#4facfe'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    load_btn = widgets.Button(
        description="📥 Load from Firebase",
        style={'button_color': '#00f2fe'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    clean_btn = widgets.Button(
        description="🧹 Clean Index",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    back_btn = widgets.Button(
        description="🏠 Main Menu",
        style={'button_color': '#a29bfe'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    # Add guide button
    guide_btn = widgets.Button(
        description="❓ Help",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='150px', height='50px')
    )
    guide_btn.add_class('guide-btn')
    guide_btn.on_click(lambda b: display(HTML(admin_screen_guide())))

    output = widgets.Output()

    def on_crawl_click(b):
        with output:
            clear_output()
            show_premium_status("🚀 Initializing Firebase crawler with fixed encoding...", "loading")

            def progress_callback(current, total, url):
                clear_output()
                display(create_fancy_progress_bar(current, total, f"Crawling: {url[:60]}..."))

            try:
                start_time = time.time()
                engine.crawl_site(max_pages=pages_slider.value, progress_callback=progress_callback)
                end_time = time.time()

                clear_output()
                show_premium_status(
                    f"✨ Crawling completed! {len(engine.pages)} pages saved to Firebase in {end_time-start_time:.2f}s",
                    "success"
                )

                # Display animated statistics
                display(HTML(f'''
                <div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin-top: 30px;">
                    <div class="stat-card">
                        <div class="stat-icon">📄</div>
                        <div class="stat-number">{len(engine.pages)}</div>
                        <div class="stat-label">Pages Crawled</div>
                    </div>
                    <div class="stat-card" style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);">
                        <div class="stat-icon">⏱️</div>
                        <div class="stat-number">{end_time-start_time:.1f}</div>
                        <div class="stat-label">Seconds</div>
                    </div>
                    <div class="stat-card" style="background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%);">
                        <div class="stat-icon">☁️</div>
                        <div class="stat-number">{len(engine.url_to_id)}</div>
                        <div class="stat-label">URLs in Firebase</div>
                    </div>
                </div>
                '''))

            except Exception as e:
                show_premium_status(f"❌ Error: {str(e)}", "error")

    def on_index_click(b):
        with output:
            clear_output()
            if not engine.pages:
                # Try loading from Firebase
                show_premium_status("📥 Loading pages from Firebase...", "loading")
                engine.load_pages_from_firebase()

            if not engine.pages:
                show_premium_status("⚠️ No pages found. Please crawl the site first", "warning")
                return

            show_premium_status("🔧 Building clean ASCII index...", "loading")

            try:
                start_time = time.time()
                engine.build_index()
                end_time = time.time()

                # Calculate word frequencies for visualization
                word_freq = {}
                for word, locations in engine.word_locations.items():
                    word_freq[word] = sum(freq for _, freq in locations)

                # Create interactive word cloud
                top_words = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:100])

                # Create visualization with Plotly
                words = list(top_words.keys())[:20]
                frequencies = list(top_words.values())[:20]

                fig = go.Figure([go.Bar(
                    x=frequencies,
                    y=words,
                    orientation='h',
                    marker=dict(
                        color=frequencies,
                        colorscale='Viridis',
                        showscale=True,
                        colorbar=dict(title="Frequency")
                    )
                )])

                fig.update_layout(
                    title="Top 20 Most Frequent Terms (Clean ASCII)",
                    xaxis_title="Frequency",
                    yaxis_title="Terms",
                    height=600,
                    template="plotly_dark",
                    showlegend=False
                )

                fig.show()

                show_premium_status(
                    f"✅ Clean index built successfully! {len(engine.word_locations)} valid terms indexed in {end_time-start_time:.2f}s",
                    "success"
                )

            except Exception as e:
                show_premium_status(f"❌ Error: {str(e)}", "error")

    def on_save_click(b):
        with output:
            clear_output()
            if not engine.word_locations:
                show_premium_status("⚠️ Please build index first", "warning")
                return

            show_premium_status("☁️ Saving clean index to Firebase...", "loading")

            try:
                engine.save_index_to_firebase()
                # Display save animation
                display(HTML('''
                <div style="text-align: center; margin: 30px;">
                    <div class="loading-spinner"></div>
                    <p style="color: #2d3748; margin-top: 20px; font-size: 20px;">Syncing clean data with Firebase...</p>
                </div>
                '''))

                time.sleep(2)  # Simulate cloud upload
                clear_output()
                show_premium_status("☁️ Clean index saved to Firebase successfully!", "success")

                # Show saved info
                display(HTML(f'''
                <div class="glass-card" style="text-align: center;">
                    <h3 style="color: #2d3748; margin-bottom: 20px;">Firebase Storage Info</h3>
                    <div style="display: flex; justify-content: center; gap: 30px;">
                        <div class="stat-card" style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);">
                            <div class="stat-icon">📄</div>
                            <div class="stat-label">Clean Index Data</div>
                            <div style="font-size: 18px; margin-top: 10px;">{len(engine.word_locations)} terms</div>
                        </div>
                        <div class="stat-card" style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);">
                            <div class="stat-icon">🔗</div>
                            <div class="stat-label">URL Mapping</div>
                            <div style="font-size: 18px; margin-top: 10px;">{len(engine.url_to_id)} URLs</div>
                        </div>
                    </div>
                </div>
                '''))

            except Exception as e:
                show_premium_status(f"❌ Error: {str(e)}", "error")

    def on_load_click(b):
        with output:
            clear_output()
            show_premium_status("📥 Loading data from Firebase...", "loading")

            try:
                # Load index
                engine.load_index_from_firebase()

                # Load pages
                engine.load_pages_from_firebase()

                clear_output()
                show_premium_status(
                    f"✅ Data loaded from Firebase! {len(engine.word_locations)} terms, {len(engine.pages)} pages",
                    "success"
                )

            except Exception as e:
                show_premium_status(f"❌ Error: {str(e)}", "error")

    def on_clean_click(b):
        with output:
            clear_output()
            show_premium_status("🧹 Cleaning index to remove corrupted entries...", "loading")

            try:
                original_count = len(engine.word_locations)
                engine.clean_existing_index()
                cleaned_count = len(engine.word_locations)

                show_premium_status(
                    f"✅ Index cleaned! Removed {original_count - cleaned_count} corrupted entries. {cleaned_count} valid terms remaining.",
                    "success"
                )

            except Exception as e:
                show_premium_status(f"❌ Error: {str(e)}", "error")

    crawl_btn.on_click(on_crawl_click)
    index_btn.on_click(on_index_click)
    save_btn.on_click(on_save_click)
    load_btn.on_click(on_load_click)
    clean_btn.on_click(on_clean_click)
    back_btn.on_click(lambda b: main_menu())

    display(HTML('<div class="glass-card">'))
    display(widgets.VBox([
        pages_slider,
        widgets.HBox([crawl_btn, index_btn], layout=widgets.Layout(justify_content='center', gap='20px')),
        widgets.HBox([save_btn, load_btn, clean_btn], layout=widgets.Layout(justify_content='center', gap='20px')),
        widgets.HBox([guide_btn, back_btn], layout=widgets.Layout(justify_content='center', gap='20px'))
    ]))
    display(HTML('</div>'))
    display(output)

In [ ]:
# @title Search Screen

def search_screen():
    clear_output()
    display(HTML(premium_css))

    display(HTML('''
    <div class="app-container">
        <h1 class="main-title">🔍 Smart Search</h1>
        <p class="subtitle">Firebase-Powered Search with Clean Indexing</p>
    </div>
    '''))

    search_input = widgets.Text(
        placeholder='Enter your search query...',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )
    search_input.add_class('search-input')

    search_btn = widgets.Button(
        description="🚀 Search",
        style={'button_color': '#f093fb'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    voice_btn = widgets.Button(
        description="🎤 Voice",
        style={'button_color': '#f5576c'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    back_btn = widgets.Button(
        description="🏠 Home",
        style={'button_color': '#4facfe'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    # Add guide button
    guide_btn = widgets.Button(
        description="❓ Help",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='150px', height='50px')
    )
    guide_btn.add_class('guide-btn')
    guide_btn.on_click(lambda b: display(HTML(search_screen_guide())))

    output = widgets.Output()

    def on_search(b):
        with output:
            clear_output()
            query = search_input.value.strip()

            if not query:
                show_premium_status("⚠️ Please enter a search query", "warning")
                return

            show_premium_status(f"🔍 Searching Firebase for: \"{query}\"", "loading")

            # Display search animation
            display(HTML('''
            <div style="text-align: center; margin: 30px;">
                <div class="loading-spinner"></div>
                <p style="color: #2d3748; margin-top: 20px; font-size: 20px;">Searching clean Firebase index...</p>
            </div>
            '''))

            time.sleep(1.5)  # Simulate Firebase query

            results = engine.search(query)

            clear_output()
            display(HTML(f'''
            <div class="glass-card">
                <h2 class="section-title">Search Results for "{query}"</h2>
                <p style="color: #718096; font-size: 18px;">Found {len(results)} relevant results</p>
            </div>
            '''))

            if results:
                for i, result in enumerate(results):
                    display(HTML(f'''
                    <div class="result-item" style="animation: slideIn 0.5s ease-out {i * 0.1}s both;">
                        <div style="display: flex; justify-content: space-between; align-items: start;">
                            <div style="flex: 1;">
                                <h3 class="result-title">{result['title']}</h3>
                                <a href="{result['url']}" target="_blank" class="result-url">
                                    {result['url']}
                                </a>
                                <p class="result-snippet">{result['snippet']}</p>
                                <span style="color: #718096; font-size: 14px;">Page ID: {result['id']}</span>
                            </div>
                            <div class="score-badge">
                                Score: {result['score']}
                            </div>
                        </div>
                    </div>
                    '''))
            else:
                display(HTML('''
                <div class="glass-card" style="text-align: center;">
                    <div style="font-size: 72px; margin-bottom: 20px;">😔</div>
                    <h3 style="color: #2d3748; font-size: 28px;">No Results Found</h3>
                    <p style="color: #718096; font-size: 18px;">Try different keywords or check if the search index is built</p>
                </div>
                '''))

    def on_voice_click(b):
        show_premium_status("🎤 Voice search coming soon in v4.0!", "info")

    search_btn.on_click(on_search)
    voice_btn.on_click(on_voice_click)
    back_btn.on_click(lambda b: main_menu())
    search_input.on_submit(lambda x: on_search(None))

    display(HTML('''
    <div class="glass-card">
        <div class="search-box">
    '''))

    display(widgets.HBox([
        search_input,
        search_btn,
        voice_btn,
        guide_btn,
        back_btn
    ], layout=widgets.Layout(justify_content='center', gap='15px')))

    display(HTML('</div></div>'))
    display(output)

In [ ]:
# @title sensor screen
def sensors_screen():
    clear_output()
    display(HTML(premium_css))

    display(HTML('''
    <div class="app-container">
        <h1 class="main-title">🌡️ Sensor Readings</h1>
        <p class="subtitle">Environmental Data Monitoring System</p>
    </div>
    '''))

    # Output containers
    control_output = widgets.Output()
    sensor_output = widgets.Output()
    status_output = widgets.Output()

    # Control widgets
    refresh_interval = widgets.IntSlider(
        value=5,
        min=1,
        max=30,
        step=1,
        description='Display Refresh (s):',
        style={'description_width': 'initial'}
    )

    data_points = widgets.IntSlider(
        value=20,
        min=5,
        max=100,
        step=5,
        description='Data Points:',
        style={'description_width': 'initial'}
    )

    show_indoor = widgets.Checkbox(
        value=True,
        description='Show Indoor',
        style={'description_width': 'initial'}
    )

    show_outdoor = widgets.Checkbox(
        value=True,
        description='Show Outdoor',
        style={'description_width': 'initial'}
    )

    chart_type = widgets.RadioButtons(
        options=['line', 'bar'],
        value='line',
        description='Chart Type:',
        style={'description_width': 'initial'}
    )

    refresh_btn = widgets.Button(
        description="🔄 Refresh",
        style={'button_color': '#00f2fe'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    export_btn = widgets.Button(
        description="📥 Export Data",
        style={'button_color': '#6c5ce7'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    back_btn = widgets.Button(
        description="🏠 Home",
        style={'button_color': '#a29bfe'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    guide_btn = widgets.Button(
        description="❓ Help",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='150px', height='50px')
    )
    guide_btn.add_class('guide-btn')

    # Create sensor manager
    sensor_manager = SensorDataManager()

    def update_status_display():
        with status_output:
            clear_output()

            # Get data status
            status = sensor_manager.get_data_status()

            # Display data source info
            data_source = "Firebase Data" if status["data_loaded"] and not sensor_manager.use_fake_data else "Simulated Data"
            points_available = status["points_available"]

            display(HTML(f'''
            <div class="glass-card" style="margin-top: 20px;">
                <h3 style="color: #2d3748; margin-bottom: 15px;">📊 Data Status</h3>
                <p style="color: #718096;">
                    <strong>Source:</strong> {data_source}<br>
                    <strong>Indoor Data Points:</strong> {points_available["indoor"]}<br>
                    <strong>Outdoor Data Points:</strong> {points_available["outdoor"]}<br>
                    <strong>Last Updated:</strong> {status["last_updated"] or "Never"}
                </p>
            </div>
            '''))

    def update_sensor_display():
        with sensor_output:
            clear_output()

            try:
                # Get latest data
                latest = sensor_manager.get_latest_data()
                indoor_data = latest["indoor"]
                outdoor_data = latest["outdoor"]

                # Get history data with specified number of points
                history = sensor_manager.get_history_data(data_points.value)
                indoor_history = history["indoor"]
                outdoor_history = history["outdoor"]

                # Display current readings
                display(HTML('<div class="glass-card">'))
                display(HTML('<h3 style="color: #2d3748; margin-bottom: 15px;">📡 Current Sensor Readings</h3>'))

                # Display cards in a responsive grid
                display(HTML('<div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px;">'))

                # Indoor data
                if show_indoor.value and indoor_data:
                    display(HTML(f'''
                    <div style="background: linear-gradient(135deg, rgba(240, 147, 251, 0.1), rgba(245, 87, 108, 0.1)); border-radius: 15px; padding: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                        <h4 style="color: #2d3748; margin-bottom: 15px;">🏠 Indoor Environment</h4>
                        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;">
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Temperature</div>
                                <div style="font-size: 24px; font-weight: 700; color: #f093fb;">{indoor_data.get("Temperature", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">°C</div>
                            </div>
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Humidity</div>
                                <div style="font-size: 24px; font-weight: 700; color: #4facfe;">{indoor_data.get("Humidity", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">%</div>
                            </div>
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Pressure</div>
                                <div style="font-size: 24px; font-weight: 700; color: #43e97b;">{indoor_data.get("Pressure", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">Pa</div>
                            </div>
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Distance</div>
                                <div style="font-size: 24px; font-weight: 700; color: #fa709a;">{indoor_data.get("Distance", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">mm</div>
                            </div>
                        </div>
                    </div>
                    '''))

                # Outdoor data
                if show_outdoor.value and outdoor_data:
                    display(HTML(f'''
                    <div style="background: linear-gradient(135deg, rgba(79, 172, 254, 0.1), rgba(0, 242, 254, 0.1)); border-radius: 15px; padding: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                        <h4 style="color: #2d3748; margin-bottom: 15px;">🌳 Outdoor Environment</h4>
                        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;">
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Temperature</div>
                                <div style="font-size: 24px; font-weight: 700; color: #f093fb;">{outdoor_data.get("Temperature", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">°C</div>
                            </div>
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Humidity</div>
                                <div style="font-size: 24px; font-weight: 700; color: #4facfe;">{outdoor_data.get("Humidity", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">%</div>
                            </div>
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Pressure</div>
                                <div style="font-size: 24px; font-weight: 700; color: #43e97b;">{outdoor_data.get("Pressure", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">Pa</div>
                            </div>
                            <div style="background: white; border-radius: 10px; padding: 15px; text-align: center;">
                                <div style="color: #718096; font-size: 14px;">Light</div>
                                <div style="font-size: 24px; font-weight: 700; color: #fa709a;">{outdoor_data.get("Day Light", "N/A")}</div>
                                <div style="color: #a0aec0; font-size: 12px;">Lux</div>
                            </div>
                        </div>
                    </div>
                    '''))

                display(HTML('</div>'))
                display(HTML('</div>'))

                # Display history charts
                if indoor_history or outdoor_history:
                    display(HTML('<div class="glass-card" style="margin-top: 20px;">'))
                    display(HTML('<h3 style="color: #2d3748; margin-bottom: 15px;">📈 Historical Data</h3>'))

                    # Plot temperature comparison
                    if (show_indoor.value and len(indoor_history) > 0) or (show_outdoor.value and len(outdoor_history) > 0):
                        # Create figure
                        plt.figure(figsize=(10, 6))

                        # Prepare timestamps for x-axis
                        timestamps = []
                        source = indoor_history if show_indoor.value and len(indoor_history) > 0 else outdoor_history

                        for item in source[-data_points.value:]:
                            ts = item.get('timestamp', '')
                            if isinstance(ts, str) and 'T' in ts:
                                time_part = ts.split('T')[1][:5]  # Extract HH:MM
                                timestamps.append(time_part)
                            else:
                                timestamps.append(str(ts)[:10])

                        x = range(len(timestamps))

                        # Plot indoor temperature
                        if show_indoor.value and len(indoor_history) > 0:
                            temps = [item.get('Temperature', 0) for item in indoor_history[-data_points.value:]]

                            if chart_type.value == 'line':
                                plt.plot(x, temps, 'o-', color='#f093fb', linewidth=2, markersize=6, label='Indoor Temp')
                            else:  # bar
                                plt.bar([i-0.2 for i in x], temps, width=0.4, color='#f093fb', alpha=0.7, label='Indoor Temp')

                        # Plot outdoor temperature
                        if show_outdoor.value and len(outdoor_history) > 0:
                            temps = [item.get('Temperature', 0) for item in outdoor_history[-data_points.value:]]

                            if chart_type.value == 'line':
                                plt.plot(x, temps, 's-', color='#4facfe', linewidth=2, markersize=6, label='Outdoor Temp')
                            else:  # bar
                                plt.bar([i+0.2 for i in x], temps, width=0.4, color='#4facfe', alpha=0.7, label='Outdoor Temp')

                        # Set labels
                        plt.title('Temperature Comparison', fontsize=14)
                        plt.ylabel('Temperature (°C)', fontsize=12)
                        plt.xlabel('Time', fontsize=12)

                        # Set x-ticks
                        if len(timestamps) <= 10:
                            plt.xticks(x, timestamps, rotation=45)
                        else:
                            step = len(timestamps) // 8
                            idx = list(range(0, len(timestamps), step))
                            plt.xticks([x[i] for i in idx], [timestamps[i] for i in idx], rotation=45)

                        plt.legend(loc='best')
                        plt.grid(True, alpha=0.3)
                        plt.tight_layout()

                        # Display plot
                        display(plt.gcf())
                        plt.close()

                    # Plot humidity comparison
                    if (show_indoor.value and len(indoor_history) > 0) or (show_outdoor.value and len(outdoor_history) > 0):
                        # Create figure
                        plt.figure(figsize=(10, 6))

                        # Use same timestamps as temperature plot
                        x = range(len(timestamps))

                        # Plot indoor humidity
                        if show_indoor.value and len(indoor_history) > 0:
                            humidity = [item.get('Humidity', 0) for item in indoor_history[-data_points.value:]]

                            if chart_type.value == 'line':
                                plt.plot(x, humidity, 'o-', color='#f093fb', linewidth=2, markersize=6, label='Indoor Humidity')
                            else:  # bar
                                plt.bar([i-0.2 for i in x], humidity, width=0.4, color='#f093fb', alpha=0.7, label='Indoor Humidity')

                        # Plot outdoor humidity
                        if show_outdoor.value and len(outdoor_history) > 0:
                            humidity = [item.get('Humidity', 0) for item in outdoor_history[-data_points.value:]]

                            if chart_type.value == 'line':
                                plt.plot(x, humidity, 's-', color='#4facfe', linewidth=2, markersize=6, label='Outdoor Humidity')
                            else:  # bar
                                plt.bar([i+0.2 for i in x], humidity, width=0.4, color='#4facfe', alpha=0.7, label='Outdoor Humidity')

                        # Set labels
                        plt.title('Humidity Comparison', fontsize=14)
                        plt.ylabel('Humidity (%)', fontsize=12)
                        plt.xlabel('Time', fontsize=12)

                        # Add comfort zone
                        plt.axhspan(40, 60, alpha=0.2, color='#43e97b', label='Comfort Zone')

                        # Set x-ticks
                        if len(timestamps) <= 10:
                            plt.xticks(x, timestamps, rotation=45)
                        else:
                            step = len(timestamps) // 8
                            idx = list(range(0, len(timestamps), step))
                            plt.xticks([x[i] for i in idx], [timestamps[i] for i in idx], rotation=45)

                        plt.legend(loc='best')
                        plt.grid(True, alpha=0.3)
                        plt.tight_layout()

                        # Display plot
                        display(plt.gcf())
                        plt.close()

                    display(HTML('</div>'))

            except Exception as e:
                display(HTML(f'''
                <div class="glass-card" style="border-left: 4px solid #f5576c; margin-top: 20px;">
                    <h3 style="color: #f5576c; margin-bottom: 10px;">❌ Error Displaying Sensor Data</h3>
                    <p style="color: #718096;">{str(e)}</p>
                </div>
                '''))

    def on_refresh_interval_change(change):
        sensor_manager.set_refresh_interval(change['new'])

    def on_data_points_change(change):
        sensor_manager.set_max_data_points(change['new'])
        update_sensor_display()

    def on_sensor_toggle_change(change):
        update_sensor_display()

    def on_chart_type_change(change):
        update_sensor_display()

    def on_export_click(b):
        try:
            # Get data for export
            data = sensor_manager.export_data()

            # Convert to pandas DataFrames
            indoor_df = pd.DataFrame(data["indoor"])
            outdoor_df = pd.DataFrame(data["outdoor"])

            # Display preview
            with sensor_output:
                clear_output()

                display(HTML('''
                <div class="glass-card">
                    <h3 style="color: #2d3748; margin-bottom: 15px;">📥 Data Export Preview</h3>
                    <p style="color: #718096; margin-bottom: 20px;">
                        In a production environment, this would download CSV files.
                    </p>
                </div>
                '''))

                # Show data preview
                if not indoor_df.empty:
                    display(HTML('<h4 style="margin-top: 15px;">Indoor Data Sample</h4>'))
                    display(indoor_df.head())

                if not outdoor_df.empty:
                    display(HTML('<h4 style="margin-top: 15px;">Outdoor Data Sample</h4>'))
                    display(outdoor_df.head())

                # Add back button for this view
                preview_back = widgets.Button(
                    description="← Back to Sensors",
                    style={'button_color': '#4facfe'},
                    layout=widgets.Layout(width='180px', height='40px')
                )

                preview_back.on_click(lambda b: update_sensor_display())

                display(widgets.HBox([preview_back],
                                    layout=widgets.Layout(justify_content='center', margin='20px 0')))

        except Exception as e:
            with sensor_output:
                clear_output()
                display(HTML(f'''
                <div class="glass-card" style="border-left: 4px solid #f5576c; margin-top: 20px;">
                    <h3 style="color: #f5576c; margin-bottom: 10px;">❌ Export Error</h3>
                    <p style="color: #718096;">{str(e)}</p>
                </div>
                '''))

    def on_refresh_click(b):
        update_status_display()
        update_sensor_display()

    def on_guide_click(b):
        display(HTML(sensors_screen_guide()))

    def on_back_click(b):
        main_menu()

    # Connect event handlers
    refresh_interval.observe(on_refresh_interval_change, names='value')
    data_points.observe(on_data_points_change, names='value')
    show_indoor.observe(on_sensor_toggle_change, names='value')
    show_outdoor.observe(on_sensor_toggle_change, names='value')
    chart_type.observe(on_chart_type_change, names='value')

    refresh_btn.on_click(on_refresh_click)
    export_btn.on_click(on_export_click)
    back_btn.on_click(on_back_click)
    guide_btn.on_click(on_guide_click)

    # Display control panel
    display(HTML('<div class="control-panel">'))
    display(HTML('<h3 style="color: #2d3748; margin-bottom: 15px;">🎮 Control Panel</h3>'))

    with control_output:
        display(widgets.VBox([
            widgets.HBox([refresh_interval, data_points],
                        layout=widgets.Layout(justify_content='space-between')),
            widgets.HBox([
                widgets.VBox([show_indoor, show_outdoor]),
                widgets.VBox([chart_type])
            ], layout=widgets.Layout(justify_content='space-between')),
            widgets.HBox([refresh_btn, export_btn, guide_btn, back_btn],
                        layout=widgets.Layout(justify_content='center', gap='10px'))
        ]))

    display(control_output)
    display(HTML('</div>'))

    # Show data status
    update_status_display()
    display(status_output)

    # Display sensor data
    update_sensor_display()
    display(sensor_output)

In [ ]:
# @title sensor screen(analytics)
def sensor_analytics_screen():
    clear_output()
    display(HTML(premium_css))

    display(HTML('''
    <div class="app-container">
        <h1 class="main-title">📈 Sensor Analytics</h1>
        <p class="subtitle">Environmental Data Analysis</p>
    </div>
    '''))

    # Create sensor manager
    sensor_manager = SensorDataManager()

    # Control buttons
    refresh_btn = widgets.Button(
        description="🔄 Refresh",
        style={'button_color': '#00f2fe'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    back_btn = widgets.Button(
        description="🏠 Home",
        style={'button_color': '#a29bfe'},
        layout=widgets.Layout(width='150px', height='50px')
    )

    guide_btn = widgets.Button(
        description="❓ Help",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='150px', height='50px')
    )
    guide_btn.add_class('guide-btn')

    # Display control buttons
    display(widgets.HBox([guide_btn, refresh_btn, back_btn],
                        layout=widgets.Layout(justify_content='flex-end', gap='15px')))

    # Main content area
    content_output = widgets.Output()

    def update_analytics():
        with content_output:
            clear_output()

            try:
                # Load data
                latest_data = sensor_manager.get_latest_data()
                history_data = sensor_manager.get_history_data(100)  # Get up to 100 points

                # Get statistics for indoor data
                indoor_data = history_data.get("indoor", [])

                # Calculate temperature statistics
                indoor_temps = [item.get('Temperature', 0) for item in indoor_data if 'Temperature' in item]
                if indoor_temps:
                    indoor_temp_min = min(indoor_temps)
                    indoor_temp_max = max(indoor_temps)
                    indoor_temp_avg = sum(indoor_temps) / len(indoor_temps)

                    # Calculate standard deviation
                    indoor_temp_variance = sum((t - indoor_temp_avg) ** 2 for t in indoor_temps) / len(indoor_temps)
                    indoor_temp_std = indoor_temp_variance ** 0.5
                else:
                    indoor_temp_min = indoor_temp_max = indoor_temp_avg = indoor_temp_std = "N/A"

                # Calculate humidity statistics
                indoor_humidity = [item.get('Humidity', 0) for item in indoor_data if 'Humidity' in item]
                if indoor_humidity:
                    indoor_humidity_min = min(indoor_humidity)
                    indoor_humidity_max = max(indoor_humidity)
                    indoor_humidity_avg = sum(indoor_humidity) / len(indoor_humidity)

                    # Calculate standard deviation
                    indoor_humidity_variance = sum((h - indoor_humidity_avg) ** 2 for h in indoor_humidity) / len(indoor_humidity)
                    indoor_humidity_std = indoor_humidity_variance ** 0.5
                else:
                    indoor_humidity_min = indoor_humidity_max = indoor_humidity_avg = indoor_humidity_std = "N/A"

                # Get statistics for outdoor data
                outdoor_data = history_data.get("outdoor", [])

                # Calculate temperature statistics
                outdoor_temps = [item.get('Temperature', 0) for item in outdoor_data if 'Temperature' in item]
                if outdoor_temps:
                    outdoor_temp_min = min(outdoor_temps)
                    outdoor_temp_max = max(outdoor_temps)
                    outdoor_temp_avg = sum(outdoor_temps) / len(outdoor_temps)

                    # Calculate standard deviation
                    outdoor_temp_variance = sum((t - outdoor_temp_avg) ** 2 for t in outdoor_temps) / len(outdoor_temps)
                    outdoor_temp_std = outdoor_temp_variance ** 0.5
                else:
                    outdoor_temp_min = outdoor_temp_max = outdoor_temp_avg = outdoor_temp_std = "N/A"

                # Calculate humidity statistics
                outdoor_humidity = [item.get('Humidity', 0) for item in outdoor_data if 'Humidity' in item]
                if outdoor_humidity:
                    outdoor_humidity_min = min(outdoor_humidity)
                    outdoor_humidity_max = max(outdoor_humidity)
                    outdoor_humidity_avg = sum(outdoor_humidity) / len(outdoor_humidity)

                    # Calculate standard deviation
                    outdoor_humidity_variance = sum((h - outdoor_humidity_avg) ** 2 for h in outdoor_humidity) / len(outdoor_humidity)
                    outdoor_humidity_std = outdoor_humidity_variance ** 0.5
                else:
                    outdoor_humidity_min = outdoor_humidity_max = outdoor_humidity_avg = outdoor_humidity_std = "N/A"

                # Display summary statistics
                display(HTML('''
                <div class="glass-card">
                    <h3 style="color: #2d3748; margin-bottom: 15px;">📊 Summary Statistics</h3>
                    <div style="overflow-x: auto;">
                        <table style="width: 100%; border-collapse: collapse;">
                            <thead>
                                <tr style="background-color: rgba(0,0,0,0.05);">
                                    <th style="padding: 12px; text-align: left; border-bottom: 2px solid #e2e8f0;">Metric</th>
                                    <th style="padding: 12px; text-align: center; border-bottom: 2px solid #e2e8f0;">Min</th>
                                    <th style="padding: 12px; text-align: center; border-bottom: 2px solid #e2e8f0;">Max</th>
                                    <th style="padding: 12px; text-align: center; border-bottom: 2px solid #e2e8f0;">Average</th>
                                    <th style="padding: 12px; text-align: center; border-bottom: 2px solid #e2e8f0;">Std Dev</th>
                                </tr>
                            </thead>
                            <tbody>
                '''))

                # Indoor temperature
                format_temp_min = f"{indoor_temp_min:.1f}" if isinstance(indoor_temp_min, (int, float)) else indoor_temp_min
                format_temp_max = f"{indoor_temp_max:.1f}" if isinstance(indoor_temp_max, (int, float)) else indoor_temp_max
                format_temp_avg = f"{indoor_temp_avg:.1f}" if isinstance(indoor_temp_avg, (int, float)) else indoor_temp_avg
                format_temp_std = f"{indoor_temp_std:.2f}" if isinstance(indoor_temp_std, (int, float)) else indoor_temp_std

                display(HTML(f'''
                <tr>
                    <td style="padding: 10px; border-bottom: 1px solid #e2e8f0;">
                        <span style="font-weight: 600; color: #f093fb;">Indoor Temperature (°C)</span>
                    </td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_min}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_max}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_avg}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_std}</td>
                </tr>
                '''))

                # Outdoor temperature
                format_temp_min = f"{outdoor_temp_min:.1f}" if isinstance(outdoor_temp_min, (int, float)) else outdoor_temp_min
                format_temp_max = f"{outdoor_temp_max:.1f}" if isinstance(outdoor_temp_max, (int, float)) else outdoor_temp_max
                format_temp_avg = f"{outdoor_temp_avg:.1f}" if isinstance(outdoor_temp_avg, (int, float)) else outdoor_temp_avg
                format_temp_std = f"{outdoor_temp_std:.2f}" if isinstance(outdoor_temp_std, (int, float)) else outdoor_temp_std

                display(HTML(f'''
                <tr style="background-color: rgba(0,0,0,0.02);">
                    <td style="padding: 10px; border-bottom: 1px solid #e2e8f0;">
                        <span style="font-weight: 600; color: #4facfe;">Outdoor Temperature (°C)</span>
                    </td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_min}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_max}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_avg}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_temp_std}</td>
                </tr>
                '''))

                # Indoor humidity
                format_humidity_min = f"{indoor_humidity_min:.1f}" if isinstance(indoor_humidity_min, (int, float)) else indoor_humidity_min
                format_humidity_max = f"{indoor_humidity_max:.1f}" if isinstance(indoor_humidity_max, (int, float)) else indoor_humidity_max
                format_humidity_avg = f"{indoor_humidity_avg:.1f}" if isinstance(indoor_humidity_avg, (int, float)) else indoor_humidity_avg
                format_humidity_std = f"{indoor_humidity_std:.2f}" if isinstance(indoor_humidity_std, (int, float)) else indoor_humidity_std

                display(HTML(f'''
                <tr>
                    <td style="padding: 10px; border-bottom: 1px solid #e2e8f0;">
                        <span style="font-weight: 600; color: #f093fb;">Indoor Humidity (%)</span>
                    </td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_min}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_max}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_avg}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_std}</td>
                </tr>
                '''))

                # Outdoor humidity
                format_humidity_min = f"{outdoor_humidity_min:.1f}" if isinstance(outdoor_humidity_min, (int, float)) else outdoor_humidity_min
                format_humidity_max = f"{outdoor_humidity_max:.1f}" if isinstance(outdoor_humidity_max, (int, float)) else outdoor_humidity_max
                format_humidity_avg = f"{outdoor_humidity_avg:.1f}" if isinstance(outdoor_humidity_avg, (int, float)) else outdoor_humidity_avg
                format_humidity_std = f"{outdoor_humidity_std:.2f}" if isinstance(outdoor_humidity_std, (int, float)) else outdoor_humidity_std

                display(HTML(f'''
                <tr style="background-color: rgba(0,0,0,0.02);">
                    <td style="padding: 10px; border-bottom: 1px solid #e2e8f0;">
                        <span style="font-weight: 600; color: #4facfe;">Outdoor Humidity (%)</span>
                    </td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_min}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_max}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_avg}</td>
                    <td style="padding: 10px; text-align: center; border-bottom: 1px solid #e2e8f0;">{format_humidity_std}</td>
                </tr>
                '''))

                display(HTML('''
                            </tbody>
                        </table>
                    </div>
                </div>
                '''))

                # Create visualizations - Temperature Distribution
                if indoor_temps or outdoor_temps:
                    display(HTML('''
                    <div class="glass-card" style="margin-top: 20px;">
                        <h3 style="color: #2d3748; margin-bottom: 15px;">🌡️ Temperature Distribution</h3>
                    '''))

                    plt.figure(figsize=(10, 6))

                    if indoor_temps:
                        plt.hist(indoor_temps, bins=15, alpha=0.6, color='#f093fb', label='Indoor Temperature')

                    if outdoor_temps:
                        plt.hist(outdoor_temps, bins=15, alpha=0.6, color='#4facfe', label='Outdoor Temperature')

                    plt.xlabel('Temperature (°C)', fontsize=12)
                    plt.ylabel('Frequency', fontsize=12)
                    plt.grid(True, alpha=0.3)
                    plt.legend(loc='best')

                    plt.tight_layout()
                    display(plt.gcf())
                    plt.close()

                    display(HTML('</div>'))

                # Temperature Over Time
                if indoor_temps or outdoor_temps:
                    display(HTML('''
                    <div class="glass-card" style="margin-top: 20px;">
                        <h3 style="color: #2d3748; margin-bottom: 15px;">📈 Temperature Trends</h3>
                    '''))

                    # Create plot
                    plt.figure(figsize=(10, 6))

                    # Indoor temperature trend
                    if indoor_temps:
                        x = range(len(indoor_temps))
                        plt.plot(x, indoor_temps, 'o-', color='#f093fb', linewidth=2, markersize=6, label='Indoor Temperature')

                    # Outdoor temperature trend
                    if outdoor_temps:
                        x = range(len(outdoor_temps))
                        plt.plot(x, outdoor_temps, 's-', color='#4facfe', linewidth=2, markersize=6, label='Outdoor Temperature')

                    # Calculate average values for reference lines
                    if isinstance(indoor_temp_avg, (int, float)):
                        plt.axhline(y=indoor_temp_avg, color='#f093fb', linestyle='--', alpha=0.7, label=f'Indoor Avg ({indoor_temp_avg:.1f}°C)')

                    if isinstance(outdoor_temp_avg, (int, float)):
                        plt.axhline(y=outdoor_temp_avg, color='#4facfe', linestyle='--', alpha=0.7, label=f'Outdoor Avg ({outdoor_temp_avg:.1f}°C)')

                    plt.xlabel('Time Point', fontsize=12)
                    plt.ylabel('Temperature (°C)', fontsize=12)
                    plt.grid(True, alpha=0.3)
                    plt.legend(loc='best')

                    plt.tight_layout()
                    display(plt.gcf())
                    plt.close()

                    display(HTML('</div>'))

                # Humidity analysis
                if indoor_humidity or outdoor_humidity:
                    display(HTML('''
                    <div class="glass-card" style="margin-top: 20px;">
                        <h3 style="color: #2d3748; margin-bottom: 15px;">💧 Humidity Analysis</h3>
                    '''))

                    # Create plot
                    plt.figure(figsize=(10, 6))

                    # Add comfort zone
                    plt.axhspan(40, 60, alpha=0.2, color='#43e97b', label='Comfort Zone (40-60%)')

                    # Indoor humidity
                    if indoor_humidity:
                        x = range(len(indoor_humidity))
                        plt.plot(x, indoor_humidity, 'o-', color='#f093fb', linewidth=2, markersize=6, label='Indoor Humidity')

                        # Calculate percentage in comfort zone
                        if isinstance(indoor_humidity[0], (int, float)):
                            in_comfort = sum(1 for h in indoor_humidity if 40 <= h <= 60)
                            comfort_pct = in_comfort / len(indoor_humidity) * 100
                            plt.title(f'Indoor Humidity: {comfort_pct:.1f}% of time in comfort zone', fontsize=14)

                    # Outdoor humidity
                    if outdoor_humidity:
                        x = range(len(outdoor_humidity))
                        plt.plot(x, outdoor_humidity, 's-', color='#4facfe', linewidth=2, markersize=6, label='Outdoor Humidity')

                    plt.xlabel('Time Point', fontsize=12)
                    plt.ylabel('Humidity (%)', fontsize=12)
                    plt.grid(True, alpha=0.3)
                    plt.legend(loc='best')

                    plt.tight_layout()
                    display(plt.gcf())
                    plt.close()

                    display(HTML('</div>'))

                # Pressure Analysis (if available)
                indoor_pressure = [item.get('Pressure', 0) for item in indoor_data if 'Pressure' in item]
                outdoor_pressure = [item.get('Pressure', 0) for item in outdoor_data if 'Pressure' in item]

                if indoor_pressure or outdoor_pressure:
                    display(HTML('''
                    <div class="glass-card" style="margin-top: 20px;">
                        <h3 style="color: #2d3748; margin-bottom: 15px;">🌀 Pressure Analysis</h3>
                    '''))

                    # Create plot
                    plt.figure(figsize=(10, 6))

                    # Indoor pressure
                    if indoor_pressure:
                        x = range(len(indoor_pressure))
                        plt.plot(x, indoor_pressure, 'o-', color='#f093fb', linewidth=2, label='Indoor Pressure')
                        plt.fill_between(x, indoor_pressure, alpha=0.2, color='#f093fb')

                    # Outdoor pressure
                    if outdoor_pressure:
                        x = range(len(outdoor_pressure))
                        plt.plot(x, outdoor_pressure, 's-', color='#4facfe', linewidth=2, label='Outdoor Pressure')
                        plt.fill_between(x, outdoor_pressure, alpha=0.2, color='#4facfe')

                    plt.xlabel('Time Point', fontsize=12)
                    plt.ylabel('Pressure (Pa)', fontsize=12)
                    plt.grid(True, alpha=0.3)
                    plt.legend(loc='best')

                    plt.tight_layout()
                    display(plt.gcf())
                    plt.close()

                    display(HTML('</div>'))

                # Additional sensor analysis - Light (for outdoor)
                # Use correct key 'Day Light' instead of 'DLIGHT'
                outdoor_light = [item.get('Day Light', 0) for item in outdoor_data if 'Day Light' in item]

                if outdoor_light:
                    display(HTML('''
                    <div class="glass-card" style="margin-top: 20px;">
                        <h3 style="color: #2d3748; margin-bottom: 15px;">☀️ Light Intensity Analysis</h3>
                    '''))

                    # Create plot
                    plt.figure(figsize=(10, 6))

                    x = range(len(outdoor_light))
                    plt.plot(x, outdoor_light, 'o-', color='#f9d423', linewidth=2, label='Light Intensity')
                    plt.fill_between(x, outdoor_light, alpha=0.2, color='#f9d423')

                    plt.xlabel('Time Point', fontsize=12)
                    plt.ylabel('Light Intensity (Lux)', fontsize=12)
                    plt.grid(True, alpha=0.3)

                    # Calculate statistics
                    if outdoor_light:
                        light_min = min(outdoor_light)
                        light_max = max(outdoor_light)
                        light_avg = sum(outdoor_light) / len(outdoor_light)

                        plt.title(f'Light Intensity: Min={light_min:.0f}, Max={light_max:.0f}, Avg={light_avg:.0f} Lux', fontsize=14)

                    plt.tight_layout()
                    display(plt.gcf())
                    plt.close()

                    display(HTML('</div>'))

                # Indoor vs Outdoor Correlation
                if indoor_temps and outdoor_temps:
                    # Use the minimum length of both arrays
                    min_length = min(len(indoor_temps), len(outdoor_temps))

                    if min_length > 5:  # Only show correlation if we have enough data points
                        display(HTML('''
                        <div class="glass-card" style="margin-top: 20px;">
                            <h3 style="color: #2d3748; margin-bottom: 15px;">🔄 Indoor vs Outdoor Correlation</h3>
                        '''))

                        # Create scatter plot
                        plt.figure(figsize=(10, 6))

                        # Plot correlation
                        plt.scatter(outdoor_temps[:min_length], indoor_temps[:min_length],
                                   color='#f093fb', alpha=0.7, s=50)

                        # Add regression line
                        if min_length > 1:
                            # Simple linear regression
                            x = np.array(outdoor_temps[:min_length])
                            y = np.array(indoor_temps[:min_length])

                            # Calculate slope and intercept
                            slope = ((np.mean(x) * np.mean(y)) - np.mean(x * y)) / ((np.mean(x) * np.mean(x)) - np.mean(x * x))
                            intercept = np.mean(y) - (slope * np.mean(x))

                            # Create regression line
                            x_line = np.linspace(min(x), max(x), 100)
                            y_line = slope * x_line + intercept

                            plt.plot(x_line, y_line, color='#f5576c', linewidth=2,
                                   label=f'y = {slope:.2f}x + {intercept:.2f}')

                            # Calculate correlation coefficient
                            correlation = np.corrcoef(x, y)[0, 1]

                            plt.title(f'Correlation: {correlation:.2f}', fontsize=14)

                        # Add identity line
                        min_val = min(min(indoor_temps[:min_length]), min(outdoor_temps[:min_length]))
                        max_val = max(max(indoor_temps[:min_length]), max(outdoor_temps[:min_length]))
                        plt.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5)

                        plt.xlabel('Outdoor Temperature (°C)', fontsize=12)
                        plt.ylabel('Indoor Temperature (°C)', fontsize=12)
                        plt.grid(True, alpha=0.3)
                        plt.legend(loc='best')

                        plt.tight_layout()
                        display(plt.gcf())
                        plt.close()

                        display(HTML('</div>'))

            except Exception as e:
                display(HTML(f'''
                <div class="glass-card" style="border-left: 4px solid #f5576c; margin-top: 20px;">
                    <h3 style="color: #f5576c; margin-bottom: 10px;">❌ Analytics Error</h3>
                    <p style="color: #718096;">{str(e)}</p>
                </div>
                '''))

    # Connect button actions
    refresh_btn.on_click(lambda b: update_analytics())
    back_btn.on_click(lambda b: main_menu())
    guide_btn.on_click(lambda b: display(HTML(sensor_analytics_guide())))

    # Display content
    display(content_output)

    # Initialize analytics
    update_analytics()

In [ ]:
# @title Stats Screen (Search Analytics)

def stats_screen():
    clear_output()
    display(HTML(premium_css))

    display(HTML('''
    <div class="app-container">
        <h1 class="main-title">📊 Search Analytics</h1>
        <p class="subtitle">Search Engine Performance and Insights</p>
    </div>
    '''))

    # Calculate comprehensive statistics
    num_pages = len(engine.pages)
    num_words = len(engine.word_locations)
    num_searches = len(engine.search_history)

    # Word frequency analysis
    word_frequencies = {}
    for word, locations in engine.word_locations.items():
        word_frequencies[word] = sum(freq for _, freq in locations)

    top_words = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)[:20]

    # Display statistics cards
    display(HTML(f'''
    <div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin-bottom: 40px;">
        <div class="stat-card">
            <div class="stat-icon">📄</div>
            <div class="stat-number">{num_pages}</div>
            <div class="stat-label">Pages in Firebase</div>
        </div>

        <div class="stat-card" style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);">
            <div class="stat-icon">🔤</div>
            <div class="stat-number">{num_words}</div>
            <div class="stat-label">Clean Terms</div>
        </div>

        <div class="stat-card" style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%);">
            <div class="stat-icon">🔍</div>
            <div class="stat-number">{num_searches}</div>
            <div class="stat-label">Total Searches</div>
        </div>

        <div class="stat-card" style="background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%);">
            <div class="stat-icon">⏱️</div>
            <div class="stat-number">{sum(engine.crawl_times):.1f}</div>
            <div class="stat-label">Total Crawl Time (s)</div>
        </div>
    </div>
    '''))

    if num_words > 0:
        # Create interactive visualizations

        # 1. Word Frequency Bar Chart
        words = [word[0] for word in top_words]
        freqs = [word[1] for word in top_words]

        fig1 = go.Figure([go.Bar(
            x=words,
            y=freqs,
            marker=dict(
                color=freqs,
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Frequency")
            ),
            text=freqs,
            textposition='auto',
        )])

        fig1.update_layout(
            title="Top 20 Most Frequent Terms (Clean ASCII)",
            xaxis_title="Terms",
            yaxis_title="Frequency",
            height=500,
            template="plotly_dark",
            showlegend=False,
            xaxis_tickangle=-45
        )

        fig1.show()

        # 2. Word Length Distribution
        word_lengths = [len(word) for word in word_frequencies.keys()]

        fig2 = go.Figure()
        fig2.add_trace(go.Histogram(
            x=word_lengths,
            nbinsx=20,
            marker_color='lightblue',
            opacity=0.75,
            name='Word Length'
        ))

        fig2.update_layout(
            title="Word Length Distribution (Clean Terms)",
            xaxis_title="Word Length",
            yaxis_title="Count",
            height=400,
            template="plotly_dark",
            showlegend=False,
            bargap=0.1
        )

        fig2.show()

        # 3. Firebase Storage Visualization
        firebase_data = {
            'Data Type': ['Clean Index Terms', 'Pages', 'URL Mappings', 'Search History'],
            'Count': [num_words, num_pages, len(engine.url_to_id), num_searches]
        }

        fig3 = go.Figure(data=[
            go.Pie(labels=firebase_data['Data Type'], values=firebase_data['Count'], hole=.3)
        ])

        fig3.update_layout(
            title="Firebase Storage Distribution",
            height=400,
            template="plotly_dark",
            showlegend=True
        )

        fig3.show()

        # 4. Search History Timeline
        if engine.search_history:
            search_times = [search['time'] for search in engine.search_history]
            search_queries = [search['query'] for search in engine.search_history]

            # Get last 10 searches
            recent_searches = engine.search_history[-10:]

            display(HTML('''
            <div class="glass-card">
                <h3 style="color: #2d3748; margin-bottom: 20px;">🕐 Recent Search History</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr>
                        <th style="padding: 10px; border-bottom: 2px solid #e0e0e0; text-align: left;">Time</th>
                        <th style="padding: 10px; border-bottom: 2px solid #e0e0e0; text-align: left;">Query</th>
                    </tr>
            '''))

            for search in reversed(recent_searches):
                display(HTML(f'''
                    <tr>
                        <td style="padding: 10px; border-bottom: 1px solid #e0e0e0;">{search['time'].strftime('%Y-%m-%d %H:%M:%S')}</td>
                        <td style="padding: 10px; border-bottom: 1px solid #e0e0e0;">{search['query']}</td>
                    </tr>
                '''))

            display(HTML('''
                </table>
            </div>
            '''))

        # 5. Document Coverage
        if num_pages > 0:
            # Calculate coverage statistics
            doc_word_counts = {}
            for word, locations in engine.word_locations.items():
                for page_id, freq in locations:
                    if page_id not in doc_word_counts:
                        doc_word_counts[page_id] = 0
                    doc_word_counts[page_id] += 1

            # Find pages with most terms
            top_pages = sorted(doc_word_counts.items(), key=lambda x: x[1], reverse=True)[:10]

            if top_pages:
                page_ids = [str(p[0]) for p in top_pages]
                word_counts = [p[1] for p in top_pages]

                fig4 = go.Figure([go.Bar(
                    x=page_ids,
                    y=word_counts,
                    marker=dict(
                        color=word_counts,
                        colorscale='Blues',
                        showscale=True,
                        colorbar=dict(title="Terms")
                    )
                )])

                fig4.update_layout(
                    title="Top 10 Pages by Term Coverage",
                    xaxis_title="Page ID",
                    yaxis_title="Number of Indexed Terms",
                    height=400,
                    template="plotly_dark"
                )

                fig4.show()

        # 6. Word Cloud
        if len(word_frequencies) > 20:
            # Create word cloud data
            wordcloud_text = ' '.join([word for word, freq in top_words[:50] for _ in range(freq)])

            display(HTML('''
            <div class="glass-card" style="text-align: center;">
                <h3 style="color: #2d3748; margin-bottom: 20px;">🌤️ Word Cloud</h3>
                <p style="color: #718096;">Top 50 most frequent terms in the index</p>
            </div>
            '''))

        # 7. Index Growth Timeline
        if engine.pages:
            crawl_times = []
            for page in engine.pages[:20]:  # Show first 20 pages
                if 'crawl_time' in page:
                    try:
                        crawl_times.append(datetime.fromisoformat(page['crawl_time']))
                    except:
                        pass

            if crawl_times:
                crawl_times.sort()

                fig5 = go.Figure()
                fig5.add_trace(go.Scatter(
                    x=crawl_times,
                    y=list(range(1, len(crawl_times) + 1)),
                    mode='lines+markers',
                    line=dict(color='#6c5ce7', width=3),
                    marker=dict(size=8),
                    name='Pages Crawled'
                ))

                fig5.update_layout(
                    title="Index Growth Timeline",
                    xaxis_title="Time",
                    yaxis_title="Number of Pages",
                    height=400,
                    template="plotly_dark"
                )

                fig5.show()

    else:
        display(HTML('''
        <div class="glass-card" style="text-align: center;">
            <div style="font-size: 48px; margin-bottom: 20px;">📊</div>
            <h3 style="color: #2d3748; font-size: 24px;">No Data Available</h3>
            <p style="color: #718096;">Please crawl and index the site first to see analytics</p>
        </div>
        '''))

    # Add guide button
    guide_btn = widgets.Button(
        description="❓ Help",
        style={'button_color': '#00cec9'},
        layout=widgets.Layout(width='150px', height='50px')
    )
    guide_btn.add_class('guide-btn')
    guide_btn.on_click(lambda b: display(HTML(stats_screen_guide())))

    # Back button
    back_btn = widgets.Button(
        description="🏠 Back to Home",
        style={'button_color': '#4facfe'},
        layout=widgets.Layout(width='220px', height='60px')
    )

    back_btn.on_click(lambda b: main_menu())

    display(HTML('<div style="text-align: center; margin-top: 40px;">'))
    display(widgets.HBox([guide_btn, back_btn], layout=widgets.Layout(justify_content='center', gap='20px')))
    display(HTML('</div>'))

In [ ]:
# @title Launch Application

# Application Launch Sequence
display(HTML('''
<div style="text-align: center; padding: 80px;">
    <h1 style="font-size: 64px; color: #f093fb; margin-bottom: 30px; font-weight: 900;">
        MQTT Search Engine
    </h1>
    <p style="font-size: 28px; color: #718096; margin-bottom: 40px;">
        Enhanced v4.0 - Pre-Collected Sensor Data
    </p>
    <div class="loading-spinner"></div>
    <p style="color: #718096; margin-top: 30px; font-size: 20px;">Loading Firebase data...</p>
</div>
''' + premium_css))

# Startup animation
time.sleep(2)
clear_output()

# Launch main menu
main_menu()